In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import pickle
from datetime import datetime

class DynamicClientOnboarding:
    def __init__(self):
        # Initialize with empty profile that can be updated
        self.reset_profile()
        
        self.current_card = 0
        self.total_cards = 5
        
        # Create unique card-stack interface
        self.create_interface()
        self.show_current_card()
        
        # Global client data storage (accessible by other steps)
        global current_client_profile
        current_client_profile = self.profile

    def reset_profile(self):
        """Reset or initialize client profile"""
        self.profile = {
            'client_name': '',
            'personal': {
                'age': 30,
                'dependents': 0,
                'employment': 'Employed'
            },
            'goals': [],
            'assets': [],
            'liabilities': [],
            'income_expenses': {
                'monthly_income': 0,
                'monthly_expenses': 0,
                'emergency_fund': 0
            },
            'profile_created_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'profile_updated_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    def load_existing_profile(self, profile_data):
        """Load existing client profile data"""
        if profile_data:
            self.profile.update(profile_data)
            self.profile['profile_updated_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.update_interface_with_existing_data()

    def update_interface_with_existing_data(self):
        """Update interface fields with existing data"""
        if hasattr(self, 'name_field') and self.profile['client_name']:
            self.name_field.value = self.profile['client_name']
        if hasattr(self, 'age_field'):
            self.age_field.value = self.profile['personal']['age']
        if hasattr(self, 'dependents_field'):
            self.dependents_field.value = self.profile['personal']['dependents']
        if hasattr(self, 'employment_field'):
            self.employment_field.value = self.profile['personal']['employment']

    def create_interface(self):
        """Create card-stack interface"""
        # Header with dynamic client name
        self.header = widgets.HTML()
        self.update_header()
        
        # Card container with shadow effect
        self.card_container = widgets.VBox(
            layout=widgets.Layout(
                width='600px',
                height='500px',
                margin='0 auto',
                padding='30px',
                border='2px solid #3498db',
                border_radius='15px',
                box_shadow='0 8px 16px rgba(0,0,0,0.1)',
                background_color='#f8f9fa'
            )
        )
        
        # Card navigation dots
        self.nav_dots = widgets.HBox(
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )
        
        # Action buttons
        self.btn_prev = widgets.Button(
            description='← Previous',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        self.btn_next = widgets.Button(
            description='Next →',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )
        
        # New buttons for profile management
        self.btn_save = widgets.Button(
            description='💾 Save Profile',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        self.btn_load = widgets.Button(
            description='📂 Load Profile',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        self.btn_new = widgets.Button(
            description='🆕 New Client',
            button_style='primary',
            layout=widgets.Layout(width='120px')
        )
        
        self.btn_prev.on_click(self.prev_card)
        self.btn_next.on_click(self.next_card)
        self.btn_save.on_click(self.save_profile)
        self.btn_load.on_click(self.show_load_options)
        self.btn_new.on_click(self.new_client_profile)
        
        self.button_box = widgets.HBox(
            [self.btn_prev, widgets.HTML('&nbsp;' * 10), self.btn_next],
            layout=widgets.Layout(justify_content='center')
        )
        
        self.management_box = widgets.HBox(
            [self.btn_new, self.btn_save, self.btn_load],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Profile export area
        self.profile_export = widgets.Output()
        
        # Main container
        self.main_container = widgets.VBox([
            self.header,
            self.management_box,
            self.nav_dots,
            self.card_container,
            self.button_box,
            self.output,
            self.profile_export
        ])

    def update_header(self):
        """Update header with current client name"""
        client_name = self.profile['client_name'] if self.profile['client_name'] else 'New Client'
        self.header.value = f"""
        <h1 style='text-align:center; color:#2c3e50; margin:20px;'>
            Client Discovery Portal - {client_name}
        </h1>
        """

    def create_nav_dots(self):
        """Create navigation dots indicator"""
        dots = []
        for i in range(self.total_cards):
            color = '#3498db' if i == self.current_card else '#bdc3c7'
            dot = widgets.HTML(
                value=f"<div style='width:12px; height:12px; border-radius:50%; background:{color}; margin:0 5px; display:inline-block;'></div>"
            )
            dots.append(dot)
        self.nav_dots.children = dots

    def show_current_card(self):
        """Display current card based on step"""
        self.create_nav_dots()
        
        if self.current_card == 0:
            self.show_client_info_card()
        elif self.current_card == 1:
            self.show_goals_card()
        elif self.current_card == 2:
            self.show_assets_card()
        elif self.current_card == 3:
            self.show_liabilities_card()
        elif self.current_card == 4:
            self.show_income_expenses_card()
        
        # Update button states
        self.btn_prev.disabled = (self.current_card == 0)
        self.btn_next.description = 'Complete ✓' if self.current_card == 4 else 'Next →'

    def show_client_info_card(self):
        """Card 1: Client Information with dynamic updates"""
        self.name_field = widgets.Text(
            description='Full Name:',
            value=self.profile['client_name'],
            style={'description_width': '100px'},
            layout=widgets.Layout(width='100%')
        )
        
        # Add real-time update for name
        self.name_field.observe(self.update_client_name, names='value')
        
        self.age_field = widgets.IntSlider(
            description='Age:',
            value=self.profile['personal']['age'],
            min=18,
            max=80,
            style={'description_width': '100px'}
        )
        self.dependents_field = widgets.IntSlider(
            description='Dependents:',
            value=self.profile['personal']['dependents'],
            min=0,
            max=10,
            style={'description_width': '100px'}
        )
        self.employment_field = widgets.RadioButtons(
            description='Employment:',
            options=['Employed', 'Self-Employed', 'Business Owner', 'Retired'],
            value=self.profile['personal']['employment'],
            style={'description_width': '100px'}
        )
        
        card_content = widgets.VBox([
            widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>👤 Client Information</h2>"),
            widgets.HTML("<hr style='border:1px solid #3498db;'>"),
            self.name_field,
            self.age_field,
            self.dependents_field,
            self.employment_field
        ])
        
        self.card_container.children = [card_content]

    def update_client_name(self, change):
        """Update client name in real-time"""
        self.profile['client_name'] = change['new']
        self.update_header()
        self.update_global_profile()

    def show_goals_card(self):
        """Card 2: Financial Goals"""
        self.goal_name = widgets.Text(
            description='Goal:',
            placeholder='e.g., House Purchase, Retirement'
        )
        self.goal_timeline = widgets.Dropdown(
            description='Timeline:',
            options=[('1-3 years', 'short'), ('4-7 years', 'medium'), ('8+ years', 'long')]
        )
        self.goal_amount = widgets.IntText(
            description='Amount (₹):',
            value=0
        )
        self.add_goal_btn = widgets.Button(
            description='+ Add Goal',
            button_style='warning',
            icon='plus'
        )
        self.goals_display = widgets.HTML()
        self.update_goals_display()
        
        self.add_goal_btn.on_click(self.add_goal)
        
        card_content = widgets.VBox([
            widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>🎯 Financial Goals</h2>"),
            widgets.HTML("<hr style='border:1px solid #3498db;'>"),
            widgets.HBox([self.goal_name, self.goal_timeline]),
            widgets.HBox([self.goal_amount, self.add_goal_btn]),
            widgets.HTML("<br><b>Added Goals:</b>"),
            self.goals_display
        ])
        
        self.card_container.children = [card_content]

    def show_assets_card(self):
        """Card 3: Assets"""
        self.asset_type = widgets.Dropdown(
            description='Asset Type:',
            options=['Fixed Deposit', 'Mutual Fund', 'Property', 'Gold', 'Stocks', 'Other']
        )
        self.asset_value = widgets.IntText(
            description='Value (₹):',
            value=0
        )
        self.add_asset_btn = widgets.Button(
            description='+ Add Asset',
            button_style='info',
            icon='plus'
        )
        self.assets_display = widgets.HTML()
        self.update_assets_display()
        
        self.add_asset_btn.on_click(self.add_asset)
        
        card_content = widgets.VBox([
            widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>💰 Current Assets</h2>"),
            widgets.HTML("<hr style='border:1px solid #3498db;'>"),
            widgets.HBox([self.asset_type, self.asset_value, self.add_asset_btn]),
            widgets.HTML("<br><b>Added Assets:</b>"),
            self.assets_display
        ])
        
        self.card_container.children = [card_content]

    def show_liabilities_card(self):
        """Card 4: Liabilities"""
        self.liability_type = widgets.Dropdown(
            description='Liability Type:',
            options=['Home Loan', 'Car Loan', 'Personal Loan', 'Credit Card', 'Other']
        )
        self.liability_amount = widgets.IntText(
            description='Amount (₹):',
            value=0
        )
        self.add_liability_btn = widgets.Button(
            description='+ Add Liability',
            button_style='danger',
            icon='plus'
        )
        self.liabilities_display = widgets.HTML()
        self.update_liabilities_display()
        
        self.add_liability_btn.on_click(self.add_liability)
        
        card_content = widgets.VBox([
            widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>📉 Liabilities</h2>"),
            widgets.HTML("<hr style='border:1px solid #3498db;'>"),
            widgets.HBox([self.liability_type, self.liability_amount, self.add_liability_btn]),
            widgets.HTML("<br><b>Added Liabilities:</b>"),
            self.liabilities_display
        ])
        
        self.card_container.children = [card_content]

    def show_income_expenses_card(self):
        """Card 5: Income & Expenses"""
        self.monthly_income = widgets.IntText(
            description='Monthly Income (₹):',
            value=self.profile['income_expenses']['monthly_income'],
            style={'description_width': '150px'}
        )
        self.monthly_expenses = widgets.IntText(
            description='Monthly Expenses (₹):',
            value=self.profile['income_expenses']['monthly_expenses'],
            style={'description_width': '150px'}
        )
        self.emergency_fund = widgets.IntText(
            description='Emergency Fund (₹):',
            value=self.profile['income_expenses']['emergency_fund'],
            style={'description_width': '150px'}
        )
        
        card_content = widgets.VBox([
            widgets.HTML("<h2 style='color:#2c3e50; text-align:center;'>💸 Income & Expenses</h2>"),
            widgets.HTML("<hr style='border:1px solid #3498db;'>"),
            self.monthly_income,
            self.monthly_expenses,
            self.emergency_fund,
            widgets.HTML("<br><p style='color:#7f8c8d;'><i>This helps us understand your cash flow and financial stability</i></p>")
        ])
        
        self.card_container.children = [card_content]

    def add_goal(self, btn):
        """Add goal to list and update profile"""
        if self.goal_name.value:
            goal = {
                'name': self.goal_name.value,
                'timeline': self.goal_timeline.value,
                'amount': self.goal_amount.value
            }
            self.profile['goals'].append(goal)
            self.update_goals_display()
            self.update_global_profile()
            # Clear fields
            self.goal_name.value = ''
            self.goal_amount.value = 0

    def add_asset(self, btn):
        """Add asset to list and update profile"""
        if self.asset_value.value > 0:
            asset = {
                'type': self.asset_type.value,
                'value': self.asset_value.value
            }
            self.profile['assets'].append(asset)
            self.update_assets_display()
            self.update_global_profile()
            self.asset_value.value = 0

    def add_liability(self, btn):
        """Add liability to list and update profile"""
        if self.liability_amount.value > 0:
            liability = {
                'type': self.liability_type.value,
                'amount': self.liability_amount.value
            }
            self.profile['liabilities'].append(liability)
            self.update_liabilities_display()
            self.update_global_profile()
            self.liability_amount.value = 0

    def update_goals_display(self):
        """Update goals display"""
        if self.profile['goals']:
            goals_html = ""
            for i, goal in enumerate(self.profile['goals'], 1):
                timeline_text = {'short': '1-3 years', 'medium': '4-7 years', 'long': '8+ years'}
                goals_html += f"<p>{i}. <b>{goal['name']}</b> - {timeline_text.get(goal['timeline'], goal['timeline'])} - ₹{goal['amount']:,}</p>"
            self.goals_display.value = goals_html
        else:
            self.goals_display.value = "<p><i>No goals added yet</i></p>"

    def update_assets_display(self):
        """Update assets display"""
        if self.profile['assets']:
            assets_html = ""
            total = 0
            for i, asset in enumerate(self.profile['assets'], 1):
                assets_html += f"<p>{i}. <b>{asset['type']}</b> - ₹{asset['value']:,}</p>"
                total += asset['value']
            assets_html += f"<p><b>Total Assets: ₹{total:,}</b></p>"
            self.assets_display.value = assets_html
        else:
            self.assets_display.value = "<p><i>No assets added yet</i></p>"

    def update_liabilities_display(self):
        """Update liabilities display"""
        if self.profile['liabilities']:
            liabilities_html = ""
            total = 0
            for i, liability in enumerate(self.profile['liabilities'], 1):
                liabilities_html += f"<p>{i}. <b>{liability['type']}</b> - ₹{liability['amount']:,}</p>"
                total += liability['amount']
            liabilities_html += f"<p><b>Total Liabilities: ₹{total:,}</b></p>"
            self.liabilities_display.value = liabilities_html
        else:
            self.liabilities_display.value = "<p><i>No liabilities added yet</i></p>"

    def prev_card(self, btn):
        """Go to previous card"""
        if self.current_card > 0:
            self.save_current_card_data()
            self.current_card -= 1
            self.show_current_card()

    def next_card(self, btn):
        """Go to next card or complete"""
        if self.current_card == 4:
            self.complete_onboarding()
        else:
            self.save_current_card_data()
            self.current_card += 1
            self.show_current_card()

    def save_current_card_data(self):
        """Save current card data to profile"""
        if self.current_card == 0:
            self.profile['client_name'] = self.name_field.value
            self.profile['personal'] = {
                'age': self.age_field.value,
                'dependents': self.dependents_field.value,
                'employment': self.employment_field.value
            }
        elif self.current_card == 4:
            self.profile['income_expenses'] = {
                'monthly_income': self.monthly_income.value,
                'monthly_expenses': self.monthly_expenses.value,
                'emergency_fund': self.emergency_fund.value
            }
        
        self.profile['profile_updated_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.update_global_profile()

    def update_global_profile(self):
        """Update global profile variable for access by other steps"""
        global current_client_profile
        current_client_profile = self.profile.copy()

    def save_profile(self, btn):
        """Save profile to file"""
        self.save_current_card_data()
        
        if not self.profile['client_name']:
            with self.output:
                clear_output()
                print("❌ Please enter client name before saving!")
            return
        
        filename = f"client_profile_{self.profile['client_name'].replace(' ', '_').lower()}.json"
        
        try:
            with open(filename, 'w') as f:
                json.dump(self.profile, f, indent=2)
            
            with self.output:
                clear_output()
                print(f"✅ Profile saved successfully as {filename}")
                print(f"💾 Save date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        except Exception as e:
            with self.output:
                clear_output()
                print(f"❌ Error saving profile: {str(e)}")

    def show_load_options(self, btn):
        """Show available profiles to load"""
        import os
        profile_files = [f for f in os.listdir('.') if f.startswith('client_profile_') and f.endswith('.json')]
        
        if not profile_files:
            with self.output:
                clear_output()
                print("❌ No saved profiles found!")
            return
        
        with self.output:
            clear_output()
            print("📂 Available Client Profiles:")
            for i, filename in enumerate(profile_files, 1):
                client_name = filename.replace('client_profile_', '').replace('.json', '').replace('_', ' ').title()
                print(f"{i}. {client_name} ({filename})")
            
            print("\n💡 Enter the number of the profile you want to load:")
        
        # Create load selection interface
        self.load_dropdown = widgets.Dropdown(
            options=[(f.replace('client_profile_', '').replace('.json', '').replace('_', ' ').title(), f) 
                    for f in profile_files],
            description='Select Profile:'
        )
        
        self.load_confirm_btn = widgets.Button(
            description='Load Selected Profile',
            button_style='success'
        )
        self.load_confirm_btn.on_click(self.load_selected_profile)
        
        load_interface = widgets.VBox([
            widgets.HTML("<h3>Select Profile to Load:</h3>"),
            self.load_dropdown,
            self.load_confirm_btn
        ])
        
        with self.output:
            display(load_interface)

    def load_selected_profile(self, btn):
        """Load selected profile"""
        try:
            filename = self.load_dropdown.value
            with open(filename, 'r') as f:
                loaded_profile = json.load(f)
            
            self.profile = loaded_profile
            self.profile['profile_updated_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.update_global_profile()
            self.update_header()
            
            # Reset to first card and update interface
            self.current_card = 0
            self.show_current_card()
            
            with self.output:
                clear_output()
                print(f"✅ Profile loaded successfully!")
                print(f"👤 Client: {self.profile['client_name']}")
                print(f"📅 Originally created: {self.profile.get('profile_created_date', 'Unknown')}")
                print(f"🔄 Last updated: {self.profile['profile_updated_date']}")
                
        except Exception as e:
            with self.output:
                clear_output()
                print(f"❌ Error loading profile: {str(e)}")

    def new_client_profile(self, btn):
        """Create new client profile"""
        self.reset_profile()
        self.update_global_profile()
        self.update_header()
        self.current_card = 0
        self.show_current_card()
        
        with self.output:
            clear_output()
            print("🆕 New client profile created!")
            print("Please fill in the client information.")

    def complete_onboarding(self):
        """Complete onboarding and show summary"""
        # Save final data
        self.save_current_card_data()
        
        with self.output:
            clear_output()
            print("🎉 ONBOARDING COMPLETED!")
            print("=" * 50)
            print(f"Client: {self.profile['client_name']}")
            print(f"Age: {self.profile['personal']['age']} | Dependents: {self.profile['personal']['dependents']}")
            print(f"Employment: {self.profile['personal']['employment']}")
            
            print(f"\n📊 Financial Summary:")
            print(f"Monthly Income: ₹{self.profile['income_expenses']['monthly_income']:,}")
            print(f"Monthly Expenses: ₹{self.profile['income_expenses']['monthly_expenses']:,}")
            print(f"Emergency Fund: ₹{self.profile['income_expenses']['emergency_fund']:,}")
            
            if self.profile['goals']:
                print(f"\n🎯 Goals ({len(self.profile['goals'])}):")
                for goal in self.profile['goals']:
                    print(f"  • {goal['name']} - ₹{goal['amount']:,}")
            
            total_assets = sum(asset['value'] for asset in self.profile['assets'])
            total_liabilities = sum(liability['amount'] for liability in self.profile['liabilities'])
            net_worth = total_assets - total_liabilities
            
            print(f"\n💰 Net Worth Analysis:")
            print(f"Total Assets: ₹{total_assets:,}")
            print(f"Total Liabilities: ₹{total_liabilities:,}")
            print(f"Net Worth: ₹{net_worth:,}")
            
            print(f"\n✅ Data ready for next step: Risk & Suitability Profiling")
        
        # Export current profile for easy access
        self.export_profile_for_next_steps()

    def export_profile_for_next_steps(self):
        """Export profile in format ready for next steps"""
        with self.profile_export:
            clear_output()
            print("📋 PROFILE EXPORT FOR NEXT STEPS:")
            print("=" * 40)
            print("Copy this data structure for use in subsequent steps:")
            print("\ncurrent_client_profile = {")
            print(f"    'client_name': '{self.profile['client_name']}',")
            print(f"    'personal': {self.profile['personal']},")
            print(f"    'goals': {self.profile['goals']},")
            print(f"    'assets': {self.profile['assets']},")
            print(f"    'liabilities': {self.profile['liabilities']},")
            print(f"    'income_expenses': {self.profile['income_expenses']}")
            print("}")

    def get_client_profile(self):
        """Get current client profile for use in other steps"""
        return self.profile.copy()

# Global function to get current client profile
def get_current_client_profile():
    """Get the current client profile for use in other steps"""
    try:
        global current_client_profile
        return current_client_profile
    except NameError:
        return None

# Initialize and display
print("🚀 Dynamic Client Onboarding System - Version 2.0")
print("Features: Save/Load profiles, Real-time updates, Dynamic data flow")
print("=" * 60)

onboarding = DynamicClientOnboarding()
display(onboarding.main_container)

# Instructions for next steps
print("\n💡 USAGE INSTRUCTIONS:")
print("1. Fill in client information")
print("2. Use 'Save Profile' to save client data")
print("3. Use 'Load Profile' to load existing clients")
print("4. The global variable 'current_client_profile' is updated automatically")
print("5. In next steps, use: get_current_client_profile() to access latest data")


🚀 Dynamic Client Onboarding System - Version 2.0
Features: Save/Load profiles, Real-time updates, Dynamic data flow



💡 USAGE INSTRUCTIONS:
1. Fill in client information
2. Use 'Save Profile' to save client data
3. Use 'Load Profile' to load existing clients
4. The global variable 'current_client_profile' is updated automatically
5. In next steps, use: get_current_client_profile() to access latest data


In [2]:
# Step 1: Holistic Risk & Suitability Profiling
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime

class DynamicRiskProfiler:
    def __init__(self):
        # Get live client data from Step 0
        self.client_profile = self.get_current_client_profile()
        
        self.current_question = 0
        self.total_questions = 5
        self.scores = []
        self.max_possible_score = 75  # 5 questions × 15 max points
        
        self.add_custom_css()
        self.create_interface()
        self.show_current_question()

    def get_current_client_profile(self):
        """Dynamically fetch client profile from Step 0"""
        try:
            global current_client_profile
            return current_client_profile.copy()
        except NameError:
            print("⚠️ Complete Step 0 first!")
            return {
                'client_name': 'New Client',
                'personal': {'age': 30},
                'income_expenses': {}
            }

    def add_custom_css(self):
        """Add custom styling"""
        display(HTML("""
        <style>
            .risk-question-card {
                background: #f8f9fa;
                border-radius: 10px;
                padding: 20px;
                margin: 10px 0;
                border: 1px solid #e9ecef;
            }
            .client-info-banner {
                background: linear-gradient(135deg, #667eea, #764ba2);
                color: white;
                padding: 15px;
                border-radius: 8px;
                margin-bottom: 20px;
            }
            .widget-radio-box {
                display: flex !important;
                flex-direction: column !important;
                gap: 12px !important;
            }
        </style>
        """))

    def create_interface(self):
        """Create interactive interface"""
        self.header = widgets.HTML()
        self.update_client_display()
        
        self.progress_bar = widgets.HTML()
        self.question_area = widgets.VBox()
        
        # Initialize output_messages here
        self.output_messages = widgets.Output()
        
        # Navigation buttons
        self.btn_prev = widgets.Button(description='← Previous', button_style='info')
        self.btn_next = widgets.Button(description='Next →', button_style='success')
        self.btn_prev.on_click(self.handle_prev_question)
        self.btn_next.on_click(self.handle_next_question)
        
        self.main_container = widgets.VBox([
            self.header,
            self.progress_bar,
            self.question_area,
            widgets.HBox([self.btn_prev, self.btn_next]),
            self.output_messages
        ])

    def update_client_display(self):
        """Update client info display"""
        client_name = self.client_profile.get('client_name', 'New Client')
        self.header.value = f"""
        <div class='client-info-banner'>
            <h3>{client_name}'s Risk Profiling</h3>
            <div style='display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px;'>
                <div>Age: {self.client_profile.get('personal', {}).get('age', 30)}</div>
                <div>Income: ₹{self.client_profile.get('income_expenses', {}).get('monthly_income', 0):,}</div>
                <div>Risk Profile: {self.client_profile.get('risk_profile', 'Pending')}</div>
            </div>
        </div>
        """

    def create_progress_display(self):
        """Update progress bar"""
        progress_percent = (self.current_question / self.total_questions) * 100
        self.progress_bar.value = f"""
        <div style='margin: 20px auto; text-align: center; width: 80%;'>
            <p style='margin: 0 0 5px 0; color: #6c757d; font-size: 14px;'>
                Question {self.current_question + 1} of {self.total_questions}
            </p>
            <div style='background: #e9ecef; height: 10px; border-radius: 5px;'>
                <div style='background: linear-gradient(90deg, #667eea, #764ba2); 
                           width: {progress_percent}%; height: 100%; border-radius: 5px;
                           transition: width 0.4s ease-in-out;"></div>
            </div>
        </div>"""

    def show_current_question(self):
        """Display current question"""
        if self.current_question >= self.total_questions:
            self.finalize_assessment()
            return
            
        self.create_progress_display()
        current_q = self.questions[self.current_question]
        
        question_html = f"""
        <div class='risk-question-card'>
            <h3>{current_q['title']}</h3>
            <p>{current_q['question']}</p>
        </div>
        """
        
        radio_options = [(f"{text} ({score} points)", score) 
                        for text, score in current_q['options']]
        
        self.active_radiobuttons = widgets.RadioButtons(
            options=radio_options,
            layout={'width': '100%'},
            style={'description_width': '0px'}
        )
        
        # Load previous answer if exists
        if len(self.scores) > self.current_question:
            self.active_radiobuttons.value = self.scores[self.current_question]
        
        self.question_area.children = [
            widgets.HTML(question_html),
            self.active_radiobuttons
        ]
        
        self.btn_prev.disabled = (self.current_question == 0)
        self.btn_next.description = 'Complete ✓' if self.current_question == self.total_questions - 1 else 'Next →'

    def handle_prev_question(self, btn):
        """Navigate to previous question"""
        if self.current_question > 0:
            self.current_question -= 1
            with self.output_messages:
                clear_output()
            self.show_current_question()

    def handle_next_question(self, btn):
        """Navigate to next question"""
        selected = self.active_radiobuttons.value
        if not selected:
            with self.output_messages:
                clear_output()
                print("⚠️ Please select an answer before proceeding.")
            return
        
        if len(self.scores) > self.current_question:
            self.scores[self.current_question] = selected
        else:
            self.scores.append(selected)
            
        self.current_question += 1
        with self.output_messages:
            clear_output()
        self.show_current_question()

    def finalize_assessment(self):
        """Calculate final risk profile"""
        total_score = sum(self.scores)
        percentage = (total_score / self.max_possible_score) * 100
        
        if percentage <= 30:
            profile = "Conservative"
        elif percentage <= 50:
            profile = "Moderate Conservative"
        elif percentage <= 70:
            profile = "Balanced"
        elif percentage <= 85:
            profile = "Growth Oriented"
        else:
            profile = "Aggressive Growth"
        
        # Update global profile
        global current_risk_profile
        current_risk_profile = {
            'risk_profile': profile,
            'score': total_score,
            'assessment_date': datetime.now().strftime("%Y-%m-%d")
        }
        
        with self.output_messages:
            clear_output()
            print(f"🎉 Risk Assessment Complete!")
            print(f"📊 Total Score: {total_score}/{self.max_possible_score}")
            print(f"📈 Risk Profile: {profile}")
            
    # Questionnaire Data
    questions = [
        {
            "title": "Investment Horizon",
            "question": "What is your primary investment timeframe?",
            "options": [
                ("Less than 3 years", 5),
                ("3-7 years", 8),
                ("8-15 years", 12),
                ("More than 15 years", 15)
            ]
        },
        {
            "title": "Market Volatility", 
            "question": "If your portfolio dropped 25%, what would you do?",
            "options": [
                ("Sell immediately", 3),
                ("Sell some investments", 6),
                ("Hold and wait", 12),
                ("Buy more", 15)
            ]
        },
        {
            "title": "Risk vs Return",
            "question": "Which scenario matches your comfort level?",
            "options": [
                ("Guaranteed 5% return", 3),
                ("8% return with 5% risk", 8),
                ("12% return with 15% risk", 12),
                ("18% return with 30% risk", 15)
            ]
        },
        {
            "title": "Financial Capacity",
            "question": "How would losses impact your lifestyle?",
            "options": [
                ("Severe impact", 3),
                ("Require adjustments", 7),
                ("Manageable", 12),
                ("No impact", 15)
            ]
        },
        {
            "title": "Investment Experience",
            "question": "Describe your investment knowledge?",
            "options": [
                ("Beginner", 5),
                ("Basic", 8),
                ("Intermediate", 12),
                ("Advanced", 15)
            ]
        }
    ]

# Initialize and display
risk_profiler = DynamicRiskProfiler()
display(risk_profiler.main_container)


In [3]:
# Step 2: Intelligent Market & Universe Curation
import yfinance as yf
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time
import warnings
warnings.filterwarnings('ignore')

class ComprehensiveMarketAnalyzer:
    def __init__(self):
        # Get live client data from previous steps
        self.client_profile_data = self.get_current_client_profile() # From Step 0
        self.risk_profile_data = self.get_current_risk_profile()     # From Step 1
        
        self.companies_data = []
        self.nifty_50_tickers = self.get_nifty50_tickers()
        
        # Create interface
        self.create_interface()
        self.update_client_display() # Initial display update

    def get_current_client_profile(self):
        """Dynamically fetch client profile from Step 0"""
        try:
            global current_client_profile # Expected from Step 0
            if isinstance(current_client_profile, dict):
                return current_client_profile.copy()
            else: # Fallback if not a dict
                print("⚠️ Step 0: current_client_profile is not in the expected format. Using defaults.")
                return self._default_client_profile()
        except NameError:
            print("⚠️ Step 0: 'current_client_profile' not found. Run Step 0 first. Using defaults.")
            return self._default_client_profile()

    def _default_client_profile(self):
        return {
            'client_name': 'New Client',
            'personal': {'age': 30},
            'income_expenses': {},
            # 'risk_profile' is expected from Step 1, but Step 0 might have a placeholder
        }

    def get_current_risk_profile(self):
        """Get risk assessment from Step 1"""
        try:
            global current_risk_profile # Expected from Step 1
            if isinstance(current_risk_profile, dict):
                 # Merge risk_profile from Step 1 into client_profile_data for consistency
                if 'risk_profile' not in self.client_profile_data:
                    self.client_profile_data['risk_profile'] = current_risk_profile.get('risk_profile', 'Balanced')
                elif isinstance(self.client_profile_data.get('risk_profile'), str) and current_risk_profile.get('risk_profile'):
                     self.client_profile_data['risk_profile'] = current_risk_profile.get('risk_profile', 'Balanced')

                return current_risk_profile.copy()
            else:
                print("⚠️ Step 1: current_risk_profile is not in the expected format. Using defaults.")
                return self._default_risk_profile()
        except NameError:
            print("⚠️ Step 1: 'current_risk_profile' not found. Run Step 1 first. Using defaults.")
            return self._default_risk_profile()

    def _default_risk_profile(self):
        # Ensure 'risk_profile' in client_profile_data is updated if default is used
        if 'risk_profile' not in self.client_profile_data:
            self.client_profile_data['risk_profile'] = 'Balanced'
        return {'risk_profile': 'Balanced', 'score': 0}


    def refresh_client_data(self):
        """Refresh client data from global scope and update display"""
        self.client_profile_data = self.get_current_client_profile()
        self.risk_profile_data = self.get_current_risk_profile() # This will also update client_profile_data['risk_profile']
        self.update_client_display()

    def get_nifty50_tickers(self):
        """Get top 50 NIFTY companies"""
        return [
            "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
            "HINDUNILVR.NS", "ITC.NS", "SBIN.NS", "BHARTIARTL.NS", "KOTAKBANK.NS",
            "LT.NS", "BAJFINANCE.NS", "HCLTECH.NS", "AXISBANK.NS", "ASIANPAINT.NS",
            "MARUTI.NS", "WIPRO.NS", "SUNPHARMA.NS", "ULTRACEMCO.NS", "ADANIENT.NS",
            "NESTLEIND.NS", "NTPC.NS", "ONGC.NS", "TATAMOTORS.NS", "JSWSTEEL.NS",
            "POWERGRID.NS", "GRASIM.NS", "BAJAJFINSV.NS", "TECHM.NS", "TITAN.NS",
            "INDUSINDBK.NS", "HINDALCO.NS", "CIPLA.NS", "DRREDDY.NS", "ADANIPORTS.NS",
            "BRITANNIA.NS", "EICHERMOT.NS", "COALINDIA.NS", "SBILIFE.NS", "BPCL.NS",
            "DIVISLAB.NS", "TATACONSUM.NS", "HEROMOTOCO.NS", "APOLLOHOSP.NS", "SHREECEM.NS",
            "BAJAJ-AUTO.NS", "UPL.NS", "M&M.NS", "IOC.NS", "VEDL.NS"
        ]

    def create_interface(self):
        """Create clean interface"""
        self.header = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
                <h1 style='color: white; text-align: center; margin: 0;'>
                    📊 NIFTY 50 Market Data & Risk Classification
                </h1>
            </div>
            """
        )
        
        self.client_info = widgets.HTML() # Will be populated by update_client_display
        
        self.btn_fetch = widgets.Button(
            description='📈 Fetch Company Data',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='40px', margin='10px')
        )
        self.btn_classify = widgets.Button(
            description='🎯 Risk Classification',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px', margin='10px'),
            disabled=True
        )
        
        self.btn_fetch.on_click(self.fetch_company_data)
        self.btn_classify.on_click(self.perform_classification)
        
        self.controls = widgets.HBox([self.btn_fetch, self.btn_classify])
        
        self.progress = widgets.IntProgress(
            min=0, max=len(self.nifty_50_tickers), value=0,
            description='Progress:',
            layout=widgets.Layout(width='70%', margin='10px 0')
        )
        
        self.output = widgets.Output()
        
        self.main_container = widgets.VBox([
            self.header,
            self.client_info,
            self.controls,
            self.progress,
            self.output
        ])

    def update_client_display(self):
        """Update client info display with latest data"""
        client_name = self.client_profile_data.get('client_name', 'New Client')
        # Use risk_profile from client_profile_data as it's updated by get_current_risk_profile
        risk_profile_text = self.client_profile_data.get('risk_profile', 'Balanced')
        age = self.client_profile_data.get('personal', {}).get('age', 30)
        
        self.client_info.value = f"""
        <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; 
                    border-left: 4px solid #667eea; margin-bottom: 20px;'>
            <h3 style='margin: 0; color: #2c3e50;'>Client: {client_name}</h3>
            <p style='margin: 5px 0; color: #7f8c8d;'>Age: {age}</p>
            <p style='margin: 5px 0; color: #7f8c8d;'>Risk Profile (from Step 1): {risk_profile_text}</p>
            <p style='margin: 5px 0; color: #7f8c8d;'>Analyzing {len(self.nifty_50_tickers)} major companies for investment suitability</p>
        </div>
        """

    def calculate_volatility(self, ticker):
        """Calculate annualized volatility"""
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1y")
            if len(hist) < 20: return np.nan
            daily_returns = hist['Close'].pct_change().dropna()
            return daily_returns.std() * np.sqrt(252)
        except Exception: return np.nan

    def get_debt_equity_ratio(self, ticker):
        """Get debt to equity ratio"""
        try:
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            if balance_sheet.empty: return np.nan
            
            latest_bs = balance_sheet.iloc[:, 0]
            total_debt = 0
            debt_keys = ['Total Debt', 'Long Term Debt', 'Short Long Term Debt', 'Current Debt']
            for key in debt_keys:
                if key in latest_bs.index and pd.notna(latest_bs[key]):
                    total_debt += latest_bs[key]
            
            equity_keys = ['Stockholders Equity', 'Total Stockholder Equity']
            total_equity = np.nan
            for key in equity_keys:
                if key in latest_bs.index and pd.notna(latest_bs[key]):
                    total_equity = latest_bs[key]; break
            
            if pd.isna(total_equity) or total_equity == 0: return np.nan
            return total_debt / total_equity
        except Exception: return np.nan

    def get_revenue_growth(self, ticker):
        """Get revenue growth rate"""
        try:
            stock = yf.Ticker(ticker)
            financials = stock.financials
            if financials.empty or financials.shape[1] < 2: return np.nan
            
            revenue_keys = ['Total Revenue', 'Operating Revenue']
            revenue_data = None
            for key in revenue_keys:
                if key in financials.index: revenue_data = financials.loc[key]; break
            
            if revenue_data is None: return np.nan
            valid_revenues = revenue_data.dropna()
            if len(valid_revenues) < 2: return np.nan
            
            current_rev, prev_rev = valid_revenues.iloc[0], valid_revenues.iloc[1]
            if prev_rev == 0: return np.nan
            return (current_rev - prev_rev) / abs(prev_rev)
        except Exception: return np.nan

    def is_financial_sector(self, ticker):
        """Check if company is in financial sector"""
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            sector = info.get('sector', '').lower()
            industry = info.get('industry', '').lower()
            financial_keywords = ['financial', 'bank', 'insurance', 'finance']
            return any(keyword in sector or keyword in industry for keyword in financial_keywords)
        except Exception: return False

    def classify_stock_risk(self, volatility, de_ratio, revenue_growth, is_financial=False):
        """Classify individual stock risk"""
        risk_score = 0
        if pd.notna(volatility): risk_score += 1 if volatility < 0.25 else 2 if volatility < 0.40 else 3
        else: risk_score += 2 # Default if no data
        
        if pd.notna(de_ratio):
            if is_financial: risk_score += 1 if de_ratio < 5 else 2 if de_ratio < 10 else 3
            else: risk_score += 1 if de_ratio < 0.5 else 2 if de_ratio < 1.5 else 3
        else: risk_score += 2 # Default if no data
        
        if pd.notna(revenue_growth): risk_score += 1 if revenue_growth > 0.15 else 2 if revenue_growth > 0 else 3
        else: risk_score += 2 # Default if no data
        
        if risk_score <= 4: return "Low Risk"
        elif risk_score <= 6: return "Medium Risk"
        else: return "High Risk"

    def fetch_company_data(self, btn):
        """Fetch comprehensive company data"""
        self.refresh_client_data() # Ensure latest client data is used
        self.btn_fetch.disabled = True
        self.btn_classify.disabled = True
        self.progress.value = 0
        self.companies_data = []

        with self.output:
            clear_output(wait=True)
            print("🔄 Fetching comprehensive data for NIFTY 50 companies...")
            print("This may take a few minutes...")
        
        for i, ticker in enumerate(self.nifty_50_tickers):
            try:
                self.progress.value = i + 1
                stock = yf.Ticker(ticker)
                info = stock.info
                
                company_name = info.get('shortName', ticker.replace('.NS', ''))
                sector = info.get('sector', 'Unknown')
                market_cap = info.get('marketCap', 0)
                
                volatility = self.calculate_volatility(ticker)
                de_ratio = self.get_debt_equity_ratio(ticker)
                revenue_growth = self.get_revenue_growth(ticker)
                is_financial = self.is_financial_sector(ticker)
                risk_class = self.classify_stock_risk(volatility, de_ratio, revenue_growth, is_financial)
                
                self.companies_data.append({
                    'Ticker': ticker.replace('.NS', ''), 'Company': company_name, 'Sector': sector,
                    'Market Cap (Cr)': market_cap/1e7 if market_cap > 0 else 0,
                    'Volatility': volatility, 'Debt/Equity': de_ratio,
                    'Revenue Growth': revenue_growth, 'Is Financial': is_financial,
                    'Risk Classification': risk_class
                })
                time.sleep(0.05) # Gentle rate limiting
            except Exception as e:
                with self.output: print(f"Error processing {ticker}: {str(e)}")
        
        self.display_company_table()
        self.btn_fetch.disabled = False
        self.btn_classify.disabled = False

        # Set global variable for Step 3
        global curated_companies
        curated_companies = pd.DataFrame(self.companies_data)


    def display_company_table(self):
        """Display company data in clean table format"""
        with self.output:
            # clear_output(wait=True) # Already cleared by fetch_company_data
            print("\n" + "="*150)
            print("📊 COMPREHENSIVE COMPANY DATA ANALYSIS")
            print("="*150)
            
            df = pd.DataFrame(self.companies_data)
            if df.empty:
                print("No company data fetched.")
                return

            header = f"{'No.':<3} {'Ticker':<12} {'Company Name':<30} {'Sector':<20} {'Market Cap':<15} {'Volatility':<12} {'D/E Ratio':<10} {'Rev Growth':<12} {'Financial':<10} {'Risk':<12}"
            print(header)
            print("-" * len(header))
            
            for i, row in df.iterrows():
                market_cap_str = f"₹{row['Market Cap (Cr)']:,.0f}" if row['Market Cap (Cr)'] > 0 else "N/A"
                volatility_str = f"{row['Volatility']:.1%}" if pd.notna(row['Volatility']) else "N/A"
                de_str = f"{row['Debt/Equity']:.2f}" if pd.notna(row['Debt/Equity']) else "N/A"
                growth_str = f"{row['Revenue Growth']:.1%}" if pd.notna(row['Revenue Growth']) else "N/A"
                financial_str = "Yes" if row['Is Financial'] else "No"
                
                print(f"{i+1:<3} {row['Ticker']:<12} {row['Company'][:28]:<30} {row['Sector'][:18]:<20} {market_cap_str:<15} {volatility_str:<12} {de_str:<10} {growth_str:<12} {financial_str:<10} {row['Risk Classification']:<12}")
            
            print("-" * len(header))
            print(f"Total Companies Analyzed: {len(df)}")
            
            risk_counts = df['Risk Classification'].value_counts()
            print(f"\n📈 RISK DISTRIBUTION SUMMARY:")
            for risk_type, count in risk_counts.items():
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0
                print(f"  {risk_type}: {count} companies ({percentage:.1f}%)")

    def perform_classification(self, btn):
        """Perform client-specific risk classification"""
        self.refresh_client_data() # Ensure latest client and risk profile data
        
        if not self.companies_data:
            with self.output:
                clear_output(wait=True)
                print("❌ No data available. Please fetch company data first by clicking 'Fetch Company Data'.")
            return
        
        client_risk_profile_text = self.client_profile_data.get('risk_profile', 'Balanced')
        client_name = self.client_profile_data.get('client_name', 'Valued Client')
        age = self.client_profile_data.get('personal', {}).get('age', 30)

        df = pd.DataFrame(self.companies_data)
        
        with self.output:
            # clear_output(wait=True) # Keep the fetched table visible
            print("\n" + "="*100)
            print("🎯 CLIENT-SPECIFIC INVESTMENT RECOMMENDATIONS")
            print("="*100)
            
            print(f"\nClient: {client_name}")
            print(f"Age: {age}")
            print(f"Risk Profile (from Step 1): {client_risk_profile_text}")
            
            if client_risk_profile_text in ['Conservative', 'Moderate Conservative']:
                primary_suitable = df[df['Risk Classification'] == 'Low Risk']
                secondary_suitable = df[df['Risk Classification'] == 'Medium Risk']
                avoid_companies = df[df['Risk Classification'] == 'High Risk']
            elif client_risk_profile_text == 'Balanced':
                primary_suitable = df[df['Risk Classification'].isin(['Low Risk', 'Medium Risk'])]
                secondary_suitable = df[df['Risk Classification'] == 'High Risk'] # For diversification, if any
                avoid_companies = pd.DataFrame() 
            else:  # Growth Oriented, Aggressive Growth
                primary_suitable = df[df['Risk Classification'].isin(['Medium Risk', 'High Risk'])]
                secondary_suitable = df[df['Risk Classification'] == 'Low Risk'] # For stability, if any
                avoid_companies = pd.DataFrame()
            
            def print_recommendations(title, companies_df):
                print(f"\n{title} ({len(companies_df)} companies):")
                print("-"*(len(title)+20))
                if not companies_df.empty:
                    for i, (_, row) in enumerate(companies_df.iterrows(), 1):
                        market_cap = f"₹{row['Market Cap (Cr)']:,.0f}Cr" if row['Market Cap (Cr)'] > 0 else "N/A"
                        volatility = f"{row['Volatility']:.1%}" if pd.notna(row['Volatility']) else "N/A"
                        print(f"{i:2d}. {row['Company'][:25]:<25} | {row['Sector'][:18]:<18} | MC: {market_cap:<12} | Vol: {volatility:<8} | {row['Risk Classification']}")
                else:
                    print("No companies found in this category.")

            print_recommendations("✅ PRIMARY RECOMMENDATIONS (Matching Risk Profile)", primary_suitable)
            if not secondary_suitable.empty:
                print_recommendations("⚠️ SECONDARY OPTIONS (For Diversification/Balance)", secondary_suitable)
            if not avoid_companies.empty:
                print_recommendations("❌ GENERALLY AVOID (Mismatch with Risk Profile)", avoid_companies)
            
            if not primary_suitable.empty:
                print(f"\n📊 SECTOR ANALYSIS (Primary Recommendations):")
                sector_counts = primary_suitable['Sector'].value_counts()
                for sector, count in sector_counts.items():
                    percentage = (count / len(primary_suitable)) * 100 if len(primary_suitable) > 0 else 0
                    print(f"  {sector}: {count} companies ({percentage:.1f}%)")
            
            print(f"\n🚀 NEXT STEPS:")
            print("-"*50)
            print("1. Discuss these curated lists with the client.")
            print("2. Select 8-12 companies primarily from 'Primary Recommendations'.")
            print("3. Ensure diversification across different sectors.")
            print("4. Consider market cap distribution (Large, Mid, Small) as per client goals.")
            print("5. Proceed to Step 3: Portfolio Construction using these curated companies.")
            
            print(f"\n✅ Data now available in global 'curated_companies' DataFrame for Step 3.")

# --- Example of how to use this Step 2 class ---
# This part would typically be in your main workflow.
# Ensure Step 0 and Step 1 have run and set their respective global variables.

# Placeholder for global variables (these should be set by previous steps)
# global current_client_profile
# global current_risk_profile
# current_client_profile = {'client_name': 'Deepa Sharma', 'personal': {'age': 24}, 'income_expenses': {}}
# current_risk_profile = {'risk_profile': 'Balanced', 'score': 50}


print("--- Initializing Step 2: Market Analysis ---")
# Create and display analyzer
analyzer = ComprehensiveMarketAnalyzer()
display(analyzer.main_container)


C:\Users\Deepa\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


--- Initializing Step 2: Market Analysis ---


In [4]:
# Step 3: Collaborative Goal-Driven Portfolio Construction (Personalized Allocation)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='ipywidgets') # To ignore slider update warnings if any

class DynamicPortfolioBuilder:
    def __init__(self):
        # Get live data from previous steps
        self.client_profile_data = self.get_current_client_profile()
        self.risk_profile_data = self.get_current_risk_profile()
        self.market_data_df = self.get_market_data() # This will be the DataFrame from Step 2
        
        self.recommended_portfolio = []
        self.total_investment = self.calculate_initial_investment()
        
        # Create interface
        self.create_interface()
        self.update_client_summary_display() # Update UI with fetched data

    def get_current_client_profile(self):
        """Dynamically fetch client profile from Step 0"""
        try:
            global current_client_profile # Expected from Step 0 (DynamicClientOnboarding)
            if isinstance(current_client_profile, dict):
                return current_client_profile.copy()
            else:
                print("⚠️ Step 0: 'current_client_profile' is not a dictionary. Using defaults.")
                return self._default_client_profile()
        except NameError:
            print("⚠️ Step 0: 'current_client_profile' not found. Run Step 0 first. Using defaults.")
            return self._default_client_profile()

    def _default_client_profile(self):
        return {
            'client_name': 'Friend',
            'personal': {'age': 25},
            'income_expenses': {'monthly_income': 50000, 'monthly_expenses': 30000},
            'goals': []
        }

    def get_current_risk_profile(self):
        """Get risk assessment from Step 1"""
        try:
            global current_risk_profile # Expected from Step 1 (DynamicRiskProfiler)
            if isinstance(current_risk_profile, dict):
                return current_risk_profile.copy()
            else:
                print("⚠️ Step 1: 'current_risk_profile' is not a dictionary. Using defaults.")
                return self._default_risk_profile()
        except NameError:
            print("⚠️ Step 1: 'current_risk_profile' not found. Run Step 1 first. Using defaults.")
            return self._default_risk_profile()
    
    def _default_risk_profile(self):
        return {'risk_profile': 'Balanced', 'score': 0}

    def get_market_data(self):
        """Get curated companies DataFrame from Step 2"""
        try:
            global curated_companies # Expected from Step 2 (ComprehensiveMarketAnalyzer)
            if isinstance(curated_companies, pd.DataFrame) and not curated_companies.empty:
                # Ensure required columns exist, add them with default if not
                required_cols = ['Company', 'Sector', 'Risk Classification', 'Market Cap (Cr)']
                for col in required_cols:
                    if col not in curated_companies.columns:
                         # Attempt to derive, e.g., 'Company' from 'Ticker' if that exists
                        if col == 'Company' and 'Ticker' in curated_companies.columns:
                            curated_companies['Company'] = curated_companies['Ticker']
                        else:
                            curated_companies[col] = "N/A" # Add with default if completely missing
                return curated_companies.copy()
            elif isinstance(curated_companies, list) and curated_companies: # If it's a list of dicts
                df = pd.DataFrame(curated_companies)
                print("⚠️ Step 2: 'curated_companies' was a list, converted to DataFrame.")
                return df
            else:
                print("⚠️ Step 2: 'curated_companies' DataFrame is empty or not found. Using sample data for portfolio construction demonstration.")
                return self._get_sample_market_data_df()
        except NameError:
            print("⚠️ Step 2: 'curated_companies' not found. Run Step 2 first. Using sample data for portfolio construction demonstration.")
            return self._get_sample_market_data_df()

    def _get_sample_market_data_df(self):
        # Fallback sample data if Step 2 data isn't available
        sample_companies = [
            {'Company': 'RELIANCE.NS', 'Ticker': 'RELIANCE.NS', 'Sector': 'Energy', 'Risk Classification': 'Medium Risk', 'Market Cap (Cr)': 1500000},
            {'Company': 'TCS.NS', 'Ticker': 'TCS.NS', 'Sector': 'Technology', 'Risk Classification': 'Low Risk', 'Market Cap (Cr)': 1300000},
            {'Company': 'HDFCBANK.NS', 'Ticker': 'HDFCBANK.NS', 'Sector': 'Financial Services', 'Risk Classification': 'Low Risk', 'Market Cap (Cr)': 800000},
            {'Company': 'INFY.NS', 'Ticker': 'INFY.NS', 'Sector': 'Technology', 'Risk Classification': 'Low Risk', 'Market Cap (Cr)': 600000},
        ]
        return pd.DataFrame(sample_companies)

    def calculate_initial_investment(self):
        """Calculate a suggested initial investment amount (e.g., 6 months of surplus income)"""
        income_data = self.client_profile_data.get('income_expenses', {})
        monthly_income = income_data.get('monthly_income', 50000)
        monthly_expenses = income_data.get('monthly_expenses', 30000)
        surplus = monthly_income - monthly_expenses
        if surplus > 0:
            initial_inv = surplus * 6
            return max(10000, initial_inv) # Ensure at least 10k
        return 100000 # Default if no surplus or data

    def create_interface(self):
        """Create child-friendly interface"""
        self.header = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #ff6b6b 0%, #4ecdc4 100%); 
                        padding: 20px; border-radius: 15px; margin-bottom: 20px;'>
                <h1 style='color: white; text-align: center; margin: 0; font-size: 28px;'>
                    🎯 Step 3: Building Your Dream Investment Basket! 🧺
                </h1>
                <p style='color: white; text-align: center; font-size: 16px; margin: 10px 0 0 0;'>
                    Let's pick the best companies for your money to grow! 🌱💰
                </p>
            </div>
            """
        )
        
        self.client_summary_html = widgets.HTML() # To be populated by update_client_summary_display
        
        self.investment_amount_slider = widgets.FloatSlider(
            value=self.total_investment, min=10000, max=5000000, step=10000,
            description='💰 Investment (₹):', style={'description_width': '150px'},
            layout=widgets.Layout(width='80%'), readout_format=',.0f'
        )
        
        self.btn_build_portfolio = widgets.Button(
            description='🎯 Build My Portfolio!', button_style='success',
            layout=widgets.Layout(width='220px', height='50px'), style={'font_weight': 'bold'}
        )
        self.btn_build_portfolio.on_click(self.build_portfolio_action)
        
        self.output_area = widgets.Output()
        self.error_output_area = widgets.Output() # For specific error messages
        
        self.main_container = widgets.VBox([
            self.header, self.client_summary_html,
            widgets.HTML("<h3 style='color: #4ecdc4;'>💰 How much money do you want to invest?</h3>"),
            self.investment_amount_slider, widgets.HTML("<br>"),
            self.btn_build_portfolio, self.error_output_area, self.output_area
        ])

    def update_client_summary_display(self):
        """Dynamically update the client summary HTML widget."""
        client_name = self.client_profile_data.get('client_name', 'Friend')
        risk_level = self.risk_profile_data.get('risk_profile', 'Balanced') # From Step 1
        age = self.client_profile_data.get('personal', {}).get('age', 25)
        
        self.client_summary_html.value = f"""
            <div style='background: #fff3cd; padding: 20px; border-radius: 10px; 
                        border-left: 5px solid #ffc107; margin-bottom: 20px;'>
                <h3 style='margin: 0; color: #856404;'>👋 Hello {client_name}!</h3>
                <p style='margin: 10px 0; color: #856404; font-size: 16px;'>
                    🎂 You are {age} years old.<br>
                    🎭 Your investment style (from Step 1): <strong>{risk_level}</strong>.<br>
                    💼 We're going to help your money grow like a tree! 🌳
                </p>
            </div>
        """
    
    def get_simple_explanation(self, company_name, sector, risk_level):
        """Get child-friendly company explanations"""
        # (Using the same logic as in paste-5.txt)
        explanations = {
            'RELIANCE': "🏭 This company makes fuel for cars and also has Jio for phones!", 'TCS': "💻 These smart people make computer programs for big companies!",
            'HDFCBANK': "🏦 This is like a big piggy bank where people save money!", 'INFY': "🌐 They help companies all over the world with technology!",
            'ICICIBANK': "🏦 Another safe place where people keep their money!", 'HINDUNILVR': "🧴 They make soaps, shampoos and ice creams we use daily!",
            'ITC': "🍪 They make biscuits, papers and many things we use!", 'BHARTIARTL': "📱 They provide phone and internet services like Airtel!",
            'MARUTI': "🚗 They make cars that families drive around!", 'ASIANPAINT': "🎨 They make colorful paints for houses and buildings!",
            'SUNPHARMA': "💊 They make medicines to keep people healthy!", 'TITAN': "⌚ They make beautiful watches and jewelry!",
            'NESTLEIND': "🍫 They make Maggi noodles and chocolates!", 'BRITANNIA': "🍪 They make biscuits and cakes we love to eat!",
            'BAJFINANCE': "💳 They help people buy things by lending money!", 'KOTAKBANK': "🏦 A trusted bank where smart people manage money!",
            'NTPC': "⚡ They make electricity to light up our homes!", 'COALINDIA': "⛏️ They dig coal from ground to make electricity!",
            'ONGC': "🛢️ They find oil and gas under the ground!", 'POWERGRID': "🔌 They carry electricity from power plants to homes!"
        }
        ticker_base = company_name.split('.')[0] # Remove .NS if present
        if ticker_base in explanations: return explanations[ticker_base]
        sector_explanations = {
            'Technology': "💻 They work with computers and make cool apps!", 'Financial Services': "🏦 They help people save and use money wisely!",
            'Healthcare': "💊 They make medicines and help people stay healthy!", 'Consumer Goods': "🛒 They make things we buy and use everyday!",
            'Energy': "⚡ They provide power and fuel for our daily needs!", 'Industrials': "🏭 They build and make big things for the country!",
            'Materials': "🔨 They make basic materials to build other things!", 'Automotive': '🚗 They make cars and bikes!',
            'Communication': '📱 They help us talk and use internet!'
        }
        return sector_explanations.get(sector, "🏢 This is a good company that helps people!")

    def get_risk_explanation(self, risk_level):
        """Simple risk explanations for children"""
        explanations = {
            'Low Risk': "🐌 Like a slow but steady turtle - safe but grows slowly.",
            'Medium Risk': "🚗 Like a car - sometimes fast, sometimes slow, but reliable.",
            'High Risk': "🎢 Like a roller coaster - exciting ups and downs, but can go high!"
        }
        return explanations.get(risk_level, "🚗 Balanced and steady growth.")

    def select_companies_for_portfolio(self):
        """Select companies based on risk profile from self.market_data_df"""
        with self.error_output_area: clear_output() # Clear previous errors

        if self.market_data_df.empty:
            with self.error_output_area:
                print("😅 Oops! No market data (from Step 2) is available. Cannot select companies.")
            return pd.DataFrame() # Return empty DataFrame

        # Ensure 'Risk Classification' column exists
        if 'Risk Classification' not in self.market_data_df.columns:
            with self.error_output_area:
                print("😅 Oops! Market data is missing 'Risk Classification'. Cannot select companies based on risk.")
            return pd.DataFrame()
            
        companies_df = self.market_data_df.copy()
        client_risk = self.risk_profile_data.get('risk_profile', 'Balanced')
        selected_companies_df = pd.DataFrame()

        if client_risk in ['Conservative', 'Moderate Conservative']:
            selected_companies_df = companies_df[companies_df['Risk Classification'] == 'Low Risk'].head(8)
        elif client_risk == 'Balanced':
            low_risk_companies = companies_df[companies_df['Risk Classification'] == 'Low Risk'].head(5)
            medium_risk_companies = companies_df[companies_df['Risk Classification'] == 'Medium Risk'].head(3)
            selected_companies_df = pd.concat([low_risk_companies, medium_risk_companies]).drop_duplicates().reset_index(drop=True)
        else:  # Growth Oriented, Aggressive Growth
            medium_risk_companies = companies_df[companies_df['Risk Classification'] == 'Medium Risk'].head(5)
            high_risk_companies = companies_df[companies_df['Risk Classification'] == 'High Risk'].head(3)
            selected_companies_df = pd.concat([medium_risk_companies, high_risk_companies]).drop_duplicates().reset_index(drop=True)
        
        # Ensure we have some companies, if not, take top ones regardless of strict risk match
        if selected_companies_df.empty or len(selected_companies_df) < 3 : # Need at least a few
            if not companies_df.empty:
                with self.error_output_area:
                    print(f"⚠️ Not enough companies strictly matching '{client_risk}'. Broadening selection...")
                selected_companies_df = companies_df.sort_values(by='Market Cap (Cr)', ascending=False).head(8) # Fallback to top 8 by market cap
            else: # This case should be caught by the initial empty check
                return pd.DataFrame()


        if selected_companies_df.empty:
             with self.error_output_area:
                print("😅 Oops! Still couldn't find suitable companies after broadening. Please check Step 2 data.")
        
        return selected_companies_df.head(8) # Limit to max 8 companies for simplicity

    def calculate_allocation_percentages(self, num_companies, client_risk_profile):
        """Calculate allocation percentages (handles num_companies = 0)"""
        if num_companies == 0:
            return []
        
        base_weights = []
        if client_risk_profile in ['Conservative', 'Moderate Conservative']:
            base_weights = [100 / num_companies] * num_companies
        elif client_risk_profile == 'Balanced':
            raw_weights = np.linspace(15, 8, num_companies) # Prefer slightly larger allocations for first few
            base_weights = (raw_weights / raw_weights.sum() * 100).tolist()
        else:  # Growth oriented, Aggressive Growth
            raw_weights = np.linspace(20, 5, num_companies) # More concentration
            base_weights = (raw_weights / raw_weights.sum() * 100).tolist()
        
        return [round(w, 1) for w in base_weights]

    def build_portfolio_action(self, btn_event):
        """Handles the button click to build and display the portfolio."""
        with self.error_output_area: clear_output()
        with self.output_area: clear_output() # Clear previous portfolio

        self.total_investment = self.investment_amount_slider.value
        self.refresh_data_for_build() # Refresh data just before building

        selected_companies_df = self.select_companies_for_portfolio()
        
        if selected_companies_df.empty:
            with self.error_output_area:
                print("😅 Oops! No companies were selected. Cannot build portfolio. Please check Step 2 data and filters.")
            return

        risk_profile_text = self.risk_profile_data.get('risk_profile', 'Balanced')
        allocations_list = self.calculate_allocation_percentages(len(selected_companies_df), risk_profile_text)

        if not allocations_list: # Should not happen if selected_companies_df is not empty
            with self.error_output_area:
                print("Error: Could not calculate allocations.")
            return

        # Prepare portfolio list for display and global export
        self.recommended_portfolio = [] # Reset
        portfolio_display_html = ""

        portfolio_display_html += "🎉" * 30 + "<br>"
        portfolio_display_html += "🌟 YOUR MAGICAL INVESTMENT BASKET IS READY! 🌟<br>"
        portfolio_display_html += "🎉" * 30 + "<br><br>"
        portfolio_display_html += f"💰 Total Money to Invest: ₹{self.total_investment:,.0f}<br>"
        portfolio_display_html += f"🎭 Your Investment Style: {risk_profile_text}<br>"
        portfolio_display_html += f"🏢 Number of Companies: {len(selected_companies_df)}<br><br>"
        portfolio_display_html += "="*60 + "<br>"
        portfolio_display_html += "🏢 YOUR COMPANY FRIENDS & HOW MUCH MONEY GOES TO EACH:<br>"
        portfolio_display_html += "="*60 + "<br>"

        current_portfolio_for_export = []

        for i, (idx, company_row) in enumerate(selected_companies_df.iterrows()):
            if i >= len(allocations_list): break # Safety break

            allocation_percent = allocations_list[i]
            investment_amount_for_stock = (allocation_percent / 100) * self.total_investment
            
            company_name = company_row.get('Company', 'N/A Company')
            sector = company_row.get('Sector', 'N/A Sector')
            stock_risk_level = company_row.get('Risk Classification', 'N/A Risk')

            explanation = self.get_simple_explanation(company_name, sector, stock_risk_level)
            risk_emoji_explanation = self.get_risk_explanation(stock_risk_level)
            
            portfolio_display_html += f"<br><b>{i+1}. 🏢 {company_name}</b><br>"
            portfolio_display_html += f"   💡 What they do: {explanation}<br>"
            portfolio_display_html += f"   🎯 Risk Level: {risk_emoji_explanation}<br>"
            portfolio_display_html += f"   💰 Your Investment: ₹{investment_amount_for_stock:,.0f} ({allocation_percent:.1f}%)<br>"
            portfolio_display_html += f"   🏭 Sector: {sector}<br>"

            current_portfolio_for_export.append({
                'company_name': company_name, 'sector': sector, 'stock_risk': stock_risk_level,
                'allocation_percent': allocation_percent, 'investment_amount': investment_amount_for_stock
            })
        
        portfolio_display_html += "<br>" + "="*60 + "<br>"
        portfolio_display_html += "📊 WHY THIS BASKET IS PERFECT FOR YOU:<br>"
        portfolio_display_html += "="*60 + "<br>"
        
        age = self.client_profile_data.get('personal', {}).get('age', 25)
        if age < 30: portfolio_display_html += "🌱 You're young! Your money has lots of time to grow like a big tree!<br>"
        elif age < 45: portfolio_display_html += "🌳 You're in your prime! Perfect time for steady growth!<br>"
        else: portfolio_display_html += "🌰 You're experienced! Time for careful and safe growth!<br>"
        
        if risk_profile_text in ['Conservative', 'Moderate Conservative']: portfolio_display_html += "🐌 We picked safe companies that grow slowly but surely!<br>🛡️ Your money is protected like a treasure in a strong castle!<br>"
        elif risk_profile_text == 'Balanced': portfolio_display_html += "⚖️ We balanced safe and exciting companies perfectly!<br>🎯 Some money grows steady, some grows faster!<br>"
        else: portfolio_display_html += "🎢 We picked exciting companies that can grow really fast!<br>💫 Higher rewards for being brave with your investments!<br>"
        
        sectors_in_portfolio = selected_companies_df['Sector'].value_counts()
        portfolio_display_html += f"<br>🌈 DIVERSIFICATION (Don't put all eggs in one basket!):<br>"
        portfolio_display_html += "🥚 We spread your money across different types of companies:<br>"
        for sec, count in sectors_in_portfolio.items(): portfolio_display_html += f"   • {sec}: {count} companies<br>"
        
        portfolio_display_html += f"<br>💡 SIMPLE TIPS FOR YOU:<br>"
        portfolio_display_html += "✅ Keep this investment for at least 3-5 years.<br>"
        portfolio_display_html += "✅ Don't worry if prices go up and down daily.<br>"
        portfolio_display_html += "✅ Your money will grow better than keeping in savings!<br>"
        portfolio_display_html += "✅ Review your basket once a year with your advisor.<br>"
        
        portfolio_display_html += f"<br>🎯 NEXT STEPS - WHAT HAPPENS NOW?<br>"
        portfolio_display_html += "1. 📝 Open a Demat account (like a digital wallet for shares).<br>"
        portfolio_display_html += "2. 💳 Transfer your money to the investment account.<br>"
        portfolio_display_html += "3. 🛒 Buy shares of these companies.<br>"
        portfolio_display_html += "4. 📱 Track your investment growth on phone apps.<br>"
        portfolio_display_html += "5. 😊 Watch your money grow over time!<br><br>"
        portfolio_display_html += f"🌟 Congratulations! You're now going to be a smart investor! 🌟<br>"

        with self.output_area:
            display(HTML(portfolio_display_html))
            if not selected_companies_df.empty:
                self.create_simple_pie_chart(selected_companies_df, allocations_list)
        
        # Set global variable for next step
        global final_portfolio_step3
        final_portfolio_step3 = {
            'total_investment': self.total_investment,
            'client_risk_profile': risk_profile_text,
            'portfolio_breakdown': current_portfolio_for_export,
            'diversification': sectors_in_portfolio.to_dict()
        }
        print(f"\n✅ Step 3 Complete. 'final_portfolio_step3' global variable is set for Step 4.")


    def create_simple_pie_chart(self, companies_df, allocations_list):
        """Create a colorful pie chart for children"""
        plt.figure(figsize=(10, 7))
        colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4', '#ffeaa7', 
                  '#dda0dd', '#98d8c8', '#f7dc6f', '#bb8fce', '#85c1e9']
        
        labels = []
        sizes = []
        
        for i, (idx, company_row) in enumerate(companies_df.iterrows()):
            if i >= len(allocations_list): break
            # Use .get with a default for 'Company'
            company_name = company_row.get('Company', f'Stock {i+1}')
            labels.append(f"{company_name}\n({allocations_list[i]:.1f}%)")
            sizes.append(allocations_list[i])
        
        if not sizes: # No data to plot
            print("No data to plot for pie chart.")
            plt.close() # Close the empty figure
            return

        plt.pie(sizes, labels=labels, colors=colors[:len(sizes)], 
                autopct='%1.1f%%', startangle=140, wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
                pctdistance=0.85)
        
        # Draw a circle at the center to make it a donut chart (more modern)
        centre_circle = plt.Circle((0,0),0.70,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        
        plt.title('🥧 Your Investment Pie Chart! 🥧', fontsize=18, fontweight='bold', color='#333', pad=20)
        plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
        plt.figtext(0.5, 0.01, '🌟 Each slice shows how much money goes to each company! 🌟', 
                   fontsize=12, style='italic', ha='center', color='#555')
        plt.tight_layout(rect=[0, 0.05, 1, 0.95]) # Adjust layout to prevent overlap
        plt.show()

    def refresh_data_for_build(self):
        """Refreshes data from global scope before building portfolio"""
        print("Refreshing data for portfolio construction...")
        self.client_profile_data = self.get_current_client_profile()
        self.risk_profile_data = self.get_current_risk_profile()
        self.market_data_df = self.get_market_data()
        self.update_client_summary_display() # Update UI in case data changed
        print("Data refreshed.")

# --- Example of how to use this Step 3 class ---
# This part would typically be in your main Jupyter Notebook cell for Step 3.
# Ensure Step 0, 1, and 2 have run and set their respective global variables.

# Example: If you were running this cell directly after Step 2,
# 'current_client_profile', 'current_risk_profile', and 'curated_companies'
# should already exist in the global scope.

print("--- Initializing Step 3: Portfolio Builder ---")
# Initialize Step 3's DynamicPortfolioBuilder. It will try to fetch global data.
portfolio_builder_instance = DynamicPortfolioBuilder()

# Display the Step 3 interface.
display(portfolio_builder_instance.main_container)

# After user interaction (adjusting slider and clicking "Build My Portfolio!"),
# the portfolio will be displayed and 'final_portfolio_step3' global var will be set.


--- Initializing Step 3: Portfolio Builder ---


Refreshing data for portfolio construction...
Data refreshed.

✅ Step 3 Complete. 'final_portfolio_step3' global variable is set for Step 4.


In [5]:
# Step 4: "Future-Proofing" Dialogue & Scenario Analysis (PortfolioStressTester)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

class PortfolioStressTester:
    def __init__(self):
        # Get live data from previous steps
        self.client_profile_data = self.get_current_client_profile()
        self.risk_profile_data = self.get_current_risk_profile()
        self.portfolio_data_step3 = self.get_final_portfolio_step3() # Portfolio from Step 3

        # Real historical stress test data from Indian markets (as in paste-6.txt)
        self.stress_scenarios_definitions = {
            'COVID-19 Pandemic (2020)': {
                'period': 'Jan 2020 - Mar 2020', 'nifty_drawdown': -38.4, 'duration_days': 69, 'recovery_days': 165,
                'sector_impacts': {'Low Risk': -25, 'Medium Risk': -38, 'High Risk': -50, 'Financial Services': -45, 'Technology': -28, 'Consumer Goods': -30, 'Energy': -40, 'Healthcare': -15, 'Industrials': -42, 'Materials': -40, 'Communication':-35, 'Automotive':-45},
                'description': 'Global pandemic caused massive market selloff. Swift fiscal and monetary response.'
            },
            'Global Financial Crisis (2008)': {
                'period': 'Jan 2008 - Oct 2008', 'nifty_drawdown': -64.2, 'duration_days': 300, 'recovery_days': 780,
                'sector_impacts': {'Low Risk': -50, 'Medium Risk': -65, 'High Risk': -80, 'Financial Services': -75, 'Technology': -52, 'Consumer Goods': -55, 'Energy': -70, 'Healthcare': -48, 'Industrials': -70, 'Materials': -72, 'Communication':-60, 'Automotive':-75},
                'description': 'US housing crisis triggered global financial meltdown. Prolonged recovery.'
            },
            'European Debt Crisis (2011)': {
                'period': 'Nov 2010 - Dec 2011', 'nifty_drawdown': -28.2, 'duration_days': 406, 'recovery_days': 487,
                'sector_impacts': {'Low Risk': -20, 'Medium Risk': -30, 'High Risk': -40, 'Financial Services': -38, 'Technology': -22, 'Consumer Goods': -25, 'Energy': -32, 'Healthcare': -19, 'Industrials': -35, 'Materials': -33, 'Communication':-28, 'Automotive':-30},
                'description': 'European sovereign debt crisis affected global markets. Contained impact.'
            }
            # Add more scenarios from paste-6.txt if needed
        }
        
        self.create_interface()
        self.update_client_summary_display()

    def get_current_client_profile(self):
        try:
            global current_client_profile
            if isinstance(current_client_profile, dict): return current_client_profile.copy()
            else: print("⚠️ Step 0: 'current_client_profile' not dict. Defaults used."); return self._default_client_profile()
        except NameError: print("⚠️ Step 0: 'current_client_profile' not found. Run Step 0. Defaults used."); return self._default_client_profile()

    def _default_client_profile(self):
        return {'client_name': 'Friend', 'personal': {'age': 25}, 'income_expenses': {}}

    def get_current_risk_profile(self):
        try:
            global current_risk_profile
            if isinstance(current_risk_profile, dict): return current_risk_profile.copy()
            else: print("⚠️ Step 1: 'current_risk_profile' not dict. Defaults used."); return self._default_risk_profile()
        except NameError: print("⚠️ Step 1: 'current_risk_profile' not found. Run Step 1. Defaults used."); return self._default_risk_profile()

    def _default_risk_profile(self):
        return {'risk_profile': 'Balanced', 'score': 0}

    def get_final_portfolio_step3(self):
        try:
            global final_portfolio_step3
            if isinstance(final_portfolio_step3, dict) and 'portfolio_breakdown' in final_portfolio_step3:
                return final_portfolio_step3.copy()
            else:
                print("⚠️ Step 3: 'final_portfolio_step3' not dict or missing breakdown. Defaults used.")
                return self._default_portfolio_data()
        except NameError:
            print("⚠️ Step 3: 'final_portfolio_step3' not found. Run Step 3. Defaults used.")
            return self._default_portfolio_data()

    def _default_portfolio_data(self):
        return {
            'total_investment': 100000,
            'client_risk_profile': 'Balanced', # Should align with self.risk_profile_data
            'portfolio_breakdown': [
                {'company_name': 'SAMPLE_STOCK_A', 'sector': 'Technology', 'stock_risk': 'Low Risk', 'allocation_percent': 50, 'investment_amount': 50000},
                {'company_name': 'SAMPLE_STOCK_B', 'sector': 'Financial Services', 'stock_risk': 'Medium Risk', 'allocation_percent': 50, 'investment_amount': 50000}
            ],
            'diversification': {'Technology': 1, 'Financial Services': 1}
        }

    def create_interface(self):
        self.header = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%); 
                        padding: 20px; border-radius: 15px; margin-bottom: 20px;'>
                <h1 style='color: white; text-align: center; margin: 0;'>
                    🛡️ Step 4: Portfolio Stress Testing & "Future-Proofing"
                </h1>
                <p style='color: white; text-align: center; font-size: 16px; margin: 10px 0 0 0;'>
                    Seeing how your investments might act in tough times, like a superhero training! 🦸‍♂️📉➡️📈
                </p>
            </div>
            """
        )
        self.client_summary_html = widgets.HTML()
        
        self.btn_run_stress_tests = widgets.Button(
            description='🔥 Run Historical Stress Tests!', button_style='danger',
            layout=widgets.Layout(width='280px', height='50px'), style={'font_weight': 'bold'}
        )
        self.btn_run_stress_tests.on_click(self.run_stress_tests_action)
        
        self.output_area = widgets.Output()
        self.error_output_area = widgets.Output()
        
        self.main_container = widgets.VBox([
            self.header, self.client_summary_html,
            widgets.HBox([self.btn_run_stress_tests], layout=widgets.Layout(justify_content='center')),
            self.error_output_area, self.output_area
        ])

    def update_client_summary_display(self):
        client_name = self.client_profile_data.get('client_name', 'Friend')
        risk_level = self.risk_profile_data.get('risk_profile', 'Balanced')
        total_investment_val = self.portfolio_data_step3.get('total_investment', 0)

        self.client_summary_html.value = f"""
        <div style='background: #fff9e6; padding: 20px; border-radius: 10px; 
                    border-left: 5px solid #ffc107; margin-bottom: 20px;'>
            <h3 style='margin: 0; color: #b38600;'>🛡️ Testing For: {client_name}</h3>
            <p style='margin: 8px 0; color: #b38600; font-size: 16px;'>
                Investment Style: <strong>{risk_level}</strong><br>
                Total Investment Basket Value: ₹{total_investment_val:,.0f}
            </p>
            <p style='margin: 8px 0; color: #b38600; font-size: 14px;'>
                We'll see how your basket of companies might have done during past big market 'oopsie' moments!
            </p>
        </div>
        """

    def calculate_portfolio_impact_per_scenario(self, scenario_definition, portfolio_breakdown):
        """
        Calculate the weighted impact on the portfolio for a given scenario.
        Uses individual stock risk and sector from the portfolio breakdown.
        """
        total_portfolio_impact_percent = 0
        
        if not portfolio_breakdown: return scenario_definition['nifty_drawdown'] # Fallback to NIFTY

        for stock_item in portfolio_breakdown:
            stock_allocation_percent = stock_item.get('allocation_percent', 0)
            stock_sector = stock_item.get('sector', 'Unknown Sector')
            # 'stock_risk' is the individual stock's classification from Step 2, e.g., 'Low Risk', 'Medium Risk'
            individual_stock_risk_level = stock_item.get('stock_risk', 'Medium Risk') 

            # Get base impact from scenario for the stock's risk level or sector
            # Prioritize stock's own risk level if defined in scenario_impacts, then sector, then general NIFTY
            base_impact_for_stock = scenario_definition['sector_impacts'].get(
                individual_stock_risk_level, # Check 'Low Risk', 'Medium Risk', 'High Risk' first
                scenario_definition['sector_impacts'].get(
                    stock_sector, # Then check specific sector impact
                    scenario_definition['nifty_drawdown'] # Fallback to general NIFTY drawdown
                )
            )
            
            # Contribution to portfolio drawdown
            stock_impact_contribution = (stock_allocation_percent / 100) * base_impact_for_stock
            total_portfolio_impact_percent += stock_impact_contribution
            
        return round(total_portfolio_impact_percent, 2)


    def run_stress_tests_action(self, btn_event):
        with self.error_output_area: clear_output()
        with self.output_area: clear_output()
        
        self.refresh_data_for_stress_test() # Refresh data

        portfolio_details = self.portfolio_data_step3.get('portfolio_breakdown', [])
        total_investment_value = self.portfolio_data_step3.get('total_investment', 0)
        client_risk_text = self.risk_profile_data.get('risk_profile', 'Balanced')
        client_name_text = self.client_profile_data.get('client_name', 'Friend')
        client_age_val = self.client_profile_data.get('personal', {}).get('age', 25)

        if not portfolio_details or total_investment_value == 0:
            with self.error_output_area:
                print("😅 Oops! We need the investment basket from Step 3 to see how it handles tricky times. Please complete Step 3 first!")
            return

        output_html = "🎢" * 25 + "<br>"
        output_html += "💥 SIMULATING MARKET EARTHQUAKES FOR YOUR PORTFOLIO! 💥<br>"
        output_html += "🎢" * 25 + "<br><br>"
        output_html += f"<strong>Client:</strong> {client_name_text}<br>"
        output_html += f"<strong>Investment Style:</strong> {client_risk_text}<br>"
        output_html += f"<strong>Total Investment:</strong> ₹{total_investment_value:,.0f}<br>"
        output_html += f"<strong>Analysis Date:</strong> {datetime.now().strftime('%B %d, %Y')}<br><br>"
        output_html += "="*80 + "<br>"
        output_html += "📜 HISTORICAL TOUGH TIMES & HOW YOUR BASKET MIGHT REACT:<br>"
        output_html += "="*80 + "<br>"

        overall_results = []

        for scenario_name, scenario_data in self.stress_scenarios_definitions.items():
            estimated_portfolio_drawdown_percent = self.calculate_portfolio_impact_per_scenario(scenario_data, portfolio_details)
            potential_loss_value = abs(estimated_portfolio_drawdown_percent / 100) * total_investment_value
            
            output_html += f"<div style='margin: 15px 0; padding:15px; border:1px solid #ddd; border-radius:8px; background-color:#f9f9f9;'>"
            output_html += f"<h4>📉 Scenario: {scenario_name}</h4>"
            output_html += f"   <em>({scenario_data['description']})</em><br>"
            output_html += f"   <b>Original Market Drop (NIFTY):</b> {scenario_data['nifty_drawdown']:.1f}%<br>"
            output_html += f"   <b>Your Estimated Basket Drop:</b> <strong style='color:red;'>{estimated_portfolio_drawdown_percent:.1f}%</strong><br>"
            output_html += f"   <b>Potential Value Dip:</b> <strong style='color:red;'>₹{potential_loss_value:,.0f}</strong> (from ₹{total_investment_value:,.0f})<br>"
            output_html += f"   <b>Bad Time Lasted:</b> {scenario_data['duration_days']} days (~{scenario_data['duration_days']//30} months)<br>"
            output_html += f"   <b>Time to Get Back to Normal:</b> {scenario_data['recovery_days']} days (~{scenario_data['recovery_days']//30} months)<br>"
            output_html += "</div>"
            
            overall_results.append({
                'scenario': scenario_name,
                'portfolio_drawdown': estimated_portfolio_drawdown_percent,
                'potential_loss': potential_loss_value,
                'recovery_days': scenario_data['recovery_days']
            })

        # Summary of results
        if overall_results:
            worst_scenario = min(overall_results, key=lambda x: x['portfolio_drawdown'])
            avg_drawdown = np.mean([res['portfolio_drawdown'] for res in overall_results])
            avg_recovery_days = np.mean([res['recovery_days'] for res in overall_results])

            output_html += "<br>" + "="*80 + "<br>"
            output_html += "💡 KEY LEARNINGS FROM THESE 'OOPSIE' TIMES:<br>"
            output_html += "="*80 + "<br>"
            output_html += f"   <b>Worst Expected Basket Drop:</b> {worst_scenario['portfolio_drawdown']:.1f}% (during {worst_scenario['scenario']}), losing about ₹{worst_scenario['potential_loss']:,.0f}.<br>"
            output_html += f"   <b>Average Expected Basket Drop:</b> {avg_drawdown:.1f}% across these tough times.<br>"
            output_html += f"   <b>Average Time to Get Better:</b> {avg_recovery_days:.0f} days (that's about {avg_recovery_days/30:.1f} months).<br><br>"

            output_html += "<strong>🤔 What this means for you, {}:</strong><br>".format(client_name_text)
            if client_age_val < 35:
                output_html += "   🚀 You're young! You have plenty of time to recover from these dips. Think of them as 'discount sales' for buying more!<br>"
            else:
                output_html += "   🧘 Even if markets go down, history shows they come back up. Patience is your superpower!<br>"
            
            if client_risk_text in ["Conservative", "Moderate Conservative"]:
                output_html += "   🛡️ Your 'safer' investment style means your basket might not fall as much as the whole market. Good job being careful!<br>"
            elif client_risk_text == "Balanced":
                 output_html += "   ⚖️ Your 'balanced' style means you'll feel some bumps, but also catch the upswings nicely. A good middle path!<br>"
            else: # Growth, Aggressive Growth
                output_html += "   🎢 Your 'adventurous' style means bigger dips but potentially bigger climbs! Hold tight during the drops for the exciting rides up!<br>"

            output_html += "<br><strong>Remember these Golden Rules for Tough Times:</strong><br>"
            output_html += "   1. 🚫 Don't Panic and Sell! (That's like jumping off a rollercoaster mid-ride).<br>"
            output_html += "   2. ⏳ Be Patient! (Good things come to those who wait for the market to calm down).<br>"
            output_html += "   3. 🎯 Focus on Your Long-Term Dreams! (These are just temporary bumps).<br>"
            output_html += "   4. 💰 If you can, Add More Money! (Buying when things are cheap is smart!).<br>"
            output_html += "<br>✨ Markets are like seasons, after winter (the drop), spring and summer (the growth) always come back! ✨<br>"

        with self.output_area:
            display(HTML(output_html))

        # Set global variable for next step
        global stress_test_results_step4
        stress_test_results_step4 = {
            'client_name': client_name_text,
            'client_risk_profile': client_risk_text,
            'total_investment': total_investment_value,
            'stress_test_scenarios_applied': self.stress_scenarios_definitions,
            'portfolio_impact_summary': overall_results
        }
        print(f"\n✅ Step 4 Complete. 'stress_test_results_step4' global variable is set for Step 5.")


    def refresh_data_for_stress_test(self):
        """Refreshes data from global scope just before running tests"""
        print("Refreshing data for stress testing...")
        self.client_profile_data = self.get_current_client_profile()
        self.risk_profile_data = self.get_current_risk_profile()
        self.portfolio_data_step3 = self.get_final_portfolio_step3()
        self.update_client_summary_display() # Update UI in case data changed
        print("Data refreshed for stress test.")


# --- Example of how to use this Step 4 class ---
# This part would typically be in your main Jupyter Notebook cell for Step 4.
# Ensure Step 0, 1, and 3 have run and set their respective global variables.

# Example: If you were running this cell directly after Step 3,
# 'current_client_profile', 'current_risk_profile', and 'final_portfolio_step3'
# should already exist in the global scope.

print("--- Initializing Step 4: Portfolio Stress Tester ---")
# Initialize Step 4's PortfolioStressTester.
stress_tester_instance = PortfolioStressTester()

# Display the Step 4 interface.
display(stress_tester_instance.main_container)

# After user clicks "Run Historical Stress Tests!",
# the analysis will be displayed and 'stress_test_results_step4' global var will be set.


--- Initializing Step 4: Portfolio Stress Tester ---


Refreshing data for stress testing...
Data refreshed for stress test.

✅ Step 4 Complete. 'stress_test_results_step4' global variable is set for Step 5.


In [6]:
# Step 5: The Interactive Recommendation Dashboard & Action Plan
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
warnings.filterwarnings('ignore', category=FutureWarning)


class ComprehensiveAdvisoryDashboard:
    def __init__(self):
        # Initialize all data by fetching from global scope (set by previous steps)
        self.client_profile_data = self._get_current_client_profile()
        self.risk_profile_data = self._get_current_risk_profile()
        self.market_universe_df = self._get_market_universe_data() # From Step 2: curated_companies
        self.portfolio_data = self._get_final_portfolio_data() # From Step 3: final_portfolio_step3
        self.stress_test_results = self._get_stress_test_results() # From Step 4: stress_test_results_step4
        
        self.add_custom_styling()
        self.create_dashboard()

    # --- Data Fetching Methods with Defaults ---
    def _get_current_client_profile(self):
        try:
            global current_client_profile
            if isinstance(current_client_profile, dict): return current_client_profile.copy()
            print("⚠️ Step 0: 'current_client_profile' not dict. Defaults used.")
            return self._default_client_profile()
        except NameError:
            print("⚠️ Step 0: 'current_client_profile' not found. Run Step 0. Defaults used.")
            return self._default_client_profile()

    def _default_client_profile(self):
        return {
            'client_name': 'Valued Client', 'personal': {'age': 30, 'dependents': 0, 'employment': 'Employed'},
            'income_expenses': {'monthly_income': 75000, 'monthly_expenses': 40000, 'emergency_fund': 300000},
            'goals': []
        }

    def _get_current_risk_profile(self):
        try:
            global current_risk_profile
            if isinstance(current_risk_profile, dict): return current_risk_profile.copy()
            print("⚠️ Step 1: 'current_risk_profile' not dict. Defaults used.")
            return self._default_risk_profile()
        except NameError:
            print("⚠️ Step 1: 'current_risk_profile' not found. Run Step 1. Defaults used.")
            return self._default_risk_profile()

    def _default_risk_profile(self):
        return {'risk_profile': 'Balanced', 'score': 50, 'percentage_score': 66.7, 'assessment_date': 'N/A'}

    def _get_market_universe_data(self): # From Step 2's curated_companies
        try:
            global curated_companies
            if isinstance(curated_companies, pd.DataFrame) and not curated_companies.empty:
                return curated_companies.copy()
            print("⚠️ Step 2: 'curated_companies' (DataFrame) is empty or not found. Using sample market data.")
            return self._default_market_universe_df()
        except NameError:
            print("⚠️ Step 2: 'curated_companies' not found. Run Step 2. Using sample market data.")
            return self._default_market_universe_df()
            
    def _default_market_universe_df(self):
        return pd.DataFrame([
            {'Ticker': 'SAMPLE1', 'Company': 'Sample Alpha Co', 'Sector': 'Technology', 'Risk Classification': 'Low Risk', 'Market Cap (Cr)': 100000},
            {'Ticker': 'SAMPLE2', 'Company': 'Sample Beta Services', 'Sector': 'Financials', 'Risk Classification': 'Medium Risk', 'Market Cap (Cr)': 50000}
        ])

    def _get_final_portfolio_data(self): # From Step 3's final_portfolio_step3
        try:
            global final_portfolio_step3
            if isinstance(final_portfolio_step3, dict) and 'portfolio_breakdown' in final_portfolio_step3:
                return final_portfolio_step3.copy()
            print("⚠️ Step 3: 'final_portfolio_step3' not dict or missing breakdown. Defaults used.")
            return self._default_final_portfolio_data()
        except NameError:
            print("⚠️ Step 3: 'final_portfolio_step3' not found. Run Step 3. Defaults used.")
            return self._default_final_portfolio_data()

    def _default_final_portfolio_data(self):
        return {
            'total_investment': 100000, 'client_risk_profile': 'Balanced',
            'portfolio_breakdown': [
                {'company_name': 'SAMPLE_A', 'sector': 'Technology', 'stock_risk': 'Low Risk', 'allocation_percent': 50, 'investment_amount': 50000},
                {'company_name': 'SAMPLE_B', 'sector': 'Financials', 'stock_risk': 'Medium Risk', 'allocation_percent': 50, 'investment_amount': 50000}
            ],
            'diversification': {'Technology': 1, 'Financials': 1}
        }

    def _get_stress_test_results(self): # From Step 4's stress_test_results_step4
        try:
            global stress_test_results_step4
            if isinstance(stress_test_results_step4, dict) and 'portfolio_impact_summary' in stress_test_results_step4:
                return stress_test_results_step4.copy()
            print("⚠️ Step 4: 'stress_test_results_step4' not dict or missing summary. Defaults used.")
            return self._default_stress_test_results()
        except NameError:
            print("⚠️ Step 4: 'stress_test_results_step4' not found. Run Step 4. Defaults used.")
            return self._default_stress_test_results()

    def _default_stress_test_results(self):
        return {
            'client_name': 'Valued Client', 'client_risk_profile': 'Balanced', 'total_investment': 100000,
            'portfolio_impact_summary': [
                {'scenario': 'Sample Crisis 1', 'portfolio_drawdown': -20, 'potential_loss': 20000, 'recovery_days': 180},
                {'scenario': 'Sample Crisis 2', 'portfolio_drawdown': -30, 'potential_loss': 30000, 'recovery_days': 360}
            ]
        }

    def add_custom_styling(self):
        custom_css = HTML("""
        <style>
        .dashboard-header { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 15px; box-shadow: 0 8px 32px rgba(0,0,0,0.1); margin-bottom: 20px; }
        .client-card { background: linear-gradient(135deg, #fdfcfb 0%, #e2d1c3 100%); padding: 20px; border-radius: 12px; box-shadow: 0 4px 16px rgba(0,0,0,0.1); margin: 10px 0; border-left: 5px solid #764ba2;}
        .portfolio-card { background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%); padding: 20px; border-radius: 12px; box-shadow: 0 4px 16px rgba(0,0,0,0.1); margin: 10px 0; border-left: 5px solid #6a11cb;}
        .metrics-card { background: #ffffff; padding: 15px; border-radius: 10px; box-shadow: 0 4px 12px rgba(0,0,0,0.08); margin: 10px 0; border: 1px solid #eee;}
        .action-card { background: linear-gradient(135deg, #c1dfc4 0%, #deecdd 100%); padding: 20px; border-radius: 12px; box-shadow: 0 4px 16px rgba(0,0,0,0.1); margin: 10px 0; border-left: 5px solid #02AAB0;}
        .widget-tab { box-shadow: 0 2px 10px rgba(0,0,0,0.1); border-radius: 8px; overflow: hidden; }
        .p-TabBar-tab { font-size: 15px !important; padding: 10px 18px !important; }
        .p-TabBar-tabIcon { margin-right: 8px !important; }
        .search-box input[type="text"] { background: white; border: 1px solid #ddd; border-radius: 20px; padding: 8px 15px; box-shadow: inset 0 1px 3px rgba(0,0,0,0.06); font-size:14px; width: calc(100% - 30px) !important; }
        .search-box .widget-text { margin-right: 10px !important; }
        table { width: 100%; border-collapse: separate; border-spacing:0; margin-top: 15px; font-size:14px; border: 1px solid #e0e0e0; border-radius: 8px; overflow:hidden; }
        th, td { padding: 10px 12px; border-bottom: 1px solid #e0e0e0; text-align:left; }
        th { background: #f8f9fa; color: #333; font-weight: 600; }
        tbody tr:last-child td { border-bottom: none; }
        tbody tr:hover { background-color: #f1f3f5; }
        h2, h3, h4 { color: #333; margin-top:0; }
        p { line-height: 1.6; color: #555; }
        </style>
        """)
        display(custom_css)
    
    def create_dashboard(self):
        self.header = widgets.HTML(
            value="""
            <div class='dashboard-header'>
                <h1 style='color: white; text-align: center; margin: 0; font-size: 30px;'>
                    🌟 Client Investment Co-Pilot Dashboard 🌟
                </h1>
                <p style='color: white; text-align: center; font-size: 17px; margin: 10px 0 0 0;'>
                    Your All-in-One Financial Advisory & Action Center
                </p>
            </div>
            """
        )
        
        self._create_client_overview_tab()
        self._create_portfolio_analysis_tab()
        self._create_market_explorer_tab()
        self._create_stress_testing_tab()
        self._create_action_plan_tab()
        
        self.main_tabs = widgets.Tab()
        self.main_tabs.children = [
            self.client_overview_tab_content, self.portfolio_analysis_tab_content,
            self.market_explorer_tab_content, self.stress_testing_tab_content,
            self.action_plan_tab_content
        ]
        
        tab_titles_with_icons = [
            ('👤 Client Profile', self.client_overview_tab_content), ('📊 Portfolio', self.portfolio_analysis_tab_content), 
            ('🔍 Market', self.market_explorer_tab_content), ('🛡️ Stress Test', self.stress_testing_tab_content),
            ('✅ Action Plan', self.action_plan_tab_content)
        ]
        for i, (title, _) in enumerate(tab_titles_with_icons):
            self.main_tabs.set_title(i, title)
        
        self.main_container = widgets.VBox([self.header, self.main_tabs])
        self.main_tabs.add_class('widget-tab')
        
    def _create_client_overview_tab(self):
        cpd = self.client_profile_data
        rpd = self.risk_profile_data
        
        client_info_html = f"""
            <div>
                <h3 style='color: #34495e;'>Personal Information</h3>
                <p><strong>Name:</strong> {cpd.get('client_name', 'N/A')}</p>
                <p><strong>Age:</strong> {cpd.get('personal', {}).get('age', 'N/A')} years</p>
                <p><strong>Dependents:</strong> {cpd.get('personal', {}).get('dependents', 'N/A')}</p>
                <p><strong>Employment:</strong> {cpd.get('personal', {}).get('employment', 'N/A')}</p>
            </div>"""
        financial_snapshot_html = f"""
            <div>
                <h3 style='color: #34495e;'>Financial Snapshot</h3>
                <p><strong>Income (Monthly):</strong> ₹{cpd.get('income_expenses', {}).get('monthly_income', 0):,}</p>
                <p><strong>Expenses (Monthly):</strong> ₹{cpd.get('income_expenses', {}).get('monthly_expenses', 0):,}</p>
                <p><strong>Surplus:</strong> ₹{cpd.get('income_expenses', {}).get('monthly_income', 0) - cpd.get('income_expenses', {}).get('monthly_expenses', 0):,}</p>
                <p><strong>Emergency Fund:</strong> ₹{cpd.get('income_expenses', {}).get('emergency_fund', 0):,}</p>
            </div>"""
        client_summary_html = f"""
            <div class='client-card'>
                <h2 style='margin-top: 0;'>Client Profile Summary</h2>
                <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                    {client_info_html} {financial_snapshot_html}
                </div>
            </div>"""
        
        risk_score_val = rpd.get('score', 0)
        risk_percentage_val = (risk_score_val / 75) * 100 if risk_score_val else rpd.get('percentage_score',0)

        risk_profile_card_html = f"""
            <div class='metrics-card'>
                <h2 style='margin-top: 0;'>Risk Profile Analysis</h2>
                <div style='display: flex; justify-content: space-between; align-items: center;'>
                    <div>
                        <h3 style='color: #e74c3c;'>Risk Classification: {rpd.get('risk_profile', 'N/A')}</h3>
                        <p><strong>Risk Score:</strong> {risk_score_val}/75 ({risk_percentage_val:.1f}%)</p>
                        <p><strong>Assessment Date:</strong> {rpd.get('assessment_date', 'N/A')}</p>
                    </div>
                    <div style='width: 120px; height: 120px; border-radius: 50%; background: conic-gradient(#e74c3c 0deg {risk_percentage_val * 3.6:.0f}deg, #ecf0f1 {risk_percentage_val * 3.6:.0f}deg 360deg); display: flex; align-items: center; justify-content: center;'>
                        <div style='width: 80px; height: 80px; border-radius: 50%; background: white; display: flex; align-items: center; justify-content: center; font-weight: bold; color: #2c3e50; font-size:18px;'>
                            {risk_percentage_val:.0f}%
                        </div>
                    </div>
                </div>
            </div>"""
        
        goals_html_list = ""
        if cpd.get('goals'):
            for goal in cpd['goals']:
                goals_html_list += f"<li><strong>{goal.get('name','N/A Goal')}:</strong> Target ₹{goal.get('amount',0):,} by {goal.get('timeline','N/A')}</li>"
        else:
            goals_html_list = "<p style='font-style: italic; color: #7f8c8d;'>No specific goals captured in Step 0.</p>"
        
        goals_card_html = f"""
            <div class='portfolio-card' style='background: linear-gradient(135deg, #f6d365 0%, #fda085 100%); border-left: 5px solid #ff8c00;'>
                <h2 style='margin-top: 0;'>Investment Goals</h2>
                <ul>{goals_html_list}</ul>
            </div>"""
        
        self.client_overview_tab_content = widgets.VBox([
            widgets.HTML(client_summary_html), widgets.HTML(risk_profile_card_html), widgets.HTML(goals_card_html)
        ])

    def _create_portfolio_analysis_tab(self):
        fpd = self.portfolio_data # final_portfolio_step3
        portfolio_breakdown = fpd.get('portfolio_breakdown', [])
        
        portfolio_summary_html = f"""
            <div class='portfolio-card'>
                <h2 style='margin-top: 0;'>Recommended Portfolio Composition</h2>
                <p><strong>Total Investment:</strong> ₹{fpd.get('total_investment', 0):,}</p>
                <p><strong>Risk Profile Applied:</strong> {fpd.get('client_risk_profile', 'N/A')}</p>
                <p><strong>Number of Holdings:</strong> {len(portfolio_breakdown)}</p>
            </div>"""
        
        table_rows_html = ""
        if portfolio_breakdown:
            for stock in portfolio_breakdown:
                risk_color = '#27ae60' if stock.get('stock_risk') == 'Low Risk' else '#f39c12' if stock.get('stock_risk') == 'Medium Risk' else '#e74c3c'
                table_rows_html += f"""
                    <tr>
                        <td>{stock.get('company_name', 'N/A')}</td> <td>{stock.get('sector', 'N/A')}</td>
                        <td style='text-align:center;'>{stock.get('allocation_percent', 0):.1f}%</td>
                        <td style='text-align:right;'>₹{stock.get('investment_amount', 0):,.0f}</td>
                        <td style='color: {risk_color}; font-weight:bold;'>{stock.get('stock_risk', 'N/A')}</td>
                    </tr>"""
        else:
            table_rows_html = "<tr><td colspan='5' style='text-align:center; font-style:italic;'>No portfolio data available from Step 3.</td></tr>"

        portfolio_table_html = f"""
            <div class='metrics-card'>
                <h3>Portfolio Holdings Detail</h3>
                <table><thead><tr><th>Company</th><th>Sector</th><th>Allocation %</th><th>Amount (₹)</th><th>Stock Risk</th></tr></thead>
                <tbody>{table_rows_html}</tbody></table>
            </div>"""
        
        sector_diversification_html = "<div class='action-card' style='background: linear-gradient(135deg, #FFC371 0%, #FF5F6D 100%); border-left: 5px solid #ff4e50;'><h3 style='margin-top:0;'>Sector Diversification</h3>"
        if fpd.get('diversification'):
            sector_diversification_html += "<div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(180px, 1fr)); gap: 10px;'>"
            for sector, count in fpd['diversification'].items():
                sector_diversification_html += f"<div style='background:white; padding:10px; border-radius:6px;'><h4 style='margin:0 0 5px 0; color:#c94343;'>{sector}</h4><p style='margin:0;'>{count} holding(s)</p></div>"
            sector_diversification_html += "</div>"
        else:
            sector_diversification_html += "<p style='font-style:italic;'>Sector diversification data not available.</p>"
        sector_diversification_html += "</div>"

        self.plot_button = widgets.Button(description='📊 Generate Portfolio Pie Chart', button_style='info', layout=widgets.Layout(width='auto', margin='10px 0'))
        self.plot_button.on_click(self._visualize_portfolio_action)
        self.plot_output_area = widgets.Output()
        
        self.portfolio_analysis_tab_content = widgets.VBox([
            widgets.HTML(portfolio_summary_html), widgets.HTML(portfolio_table_html),
            widgets.HTML(sector_diversification_html), self.plot_button, self.plot_output_area
        ])

    def _visualize_portfolio_action(self, btn_event):
        with self.plot_output_area:
            clear_output(wait=True)
            portfolio_breakdown = self.portfolio_data.get('portfolio_breakdown', [])
            if not portfolio_breakdown:
                print("No portfolio data to visualize.")
                return

            labels = [item.get('company_name', 'Unknown') for item in portfolio_breakdown]
            sizes = [item.get('allocation_percent', 0) for item in portfolio_breakdown]
            
            if not any(s > 0 for s in sizes): # Check if all sizes are zero or list is effectively empty
                print("Portfolio allocations are all zero or data is invalid for chart.")
                return

            fig, ax = plt.subplots(figsize=(10, 6))
            colors = plt.cm.Paired(np.arange(len(labels))/len(labels)) # Use a colormap
            
            wedges, texts, autotexts = ax.pie(sizes, autopct='%1.1f%%', startangle=140, colors=colors,
                                             wedgeprops={'edgecolor': 'white'})
            ax.axis('equal')
            plt.setp(autotexts, size=10, weight="bold", color="white")
            
            ax.legend(wedges, labels, title="Holdings", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1), fontsize='small')
            plt.title('Portfolio Allocation by Company', fontsize=16, fontweight='bold', pad=20)
            plt.tight_layout(rect=[0, 0, 0.8, 1]) # Adjust layout to make space for legend
            plt.show()

    def _create_market_explorer_tab(self):
        self.search_input_widget = widgets.Text(placeholder='🔍 Search by Ticker or Company Name...', layout=widgets.Layout(width='calc(100% - 120px)'))
        self.search_input_widget.add_class("search-box")
        self.search_button_widget = widgets.Button(description='Search', button_style='primary', layout=widgets.Layout(width='100px'))
        self.search_button_widget.on_click(self._search_companies_action)
        
        search_bar_ui = widgets.HBox([self.search_input_widget, self.search_button_widget], layout=widgets.Layout(width='100%', margin='10px 0'))
        
        universe_summary_html = f"""
            <div class='metrics-card'>
                <h2 style='margin-top:0;'>Market Universe Overview (from Step 2)</h2>
                <p><strong>Total Companies in Universe:</strong> {len(self.market_universe_df)}</p>
                <p><strong>Sectors Covered:</strong> {self.market_universe_df['Sector'].nunique() if not self.market_universe_df.empty else 0}</p>
            </div>"""
            
        self.company_results_display_area = widgets.HTML(value="<p style='text-align:center; font-style:italic;'>Enter search term or leave blank to see all.</p>")
        self._display_market_universe_table() # Initial display of all or top N

        self.market_explorer_tab_content = widgets.VBox([
            widgets.HTML(universe_summary_html), search_bar_ui, self.company_results_display_area
        ])
        
    def _search_companies_action(self, btn_event):
        search_term = self.search_input_widget.value.lower()
        self._display_market_universe_table(search_term)

    def _display_market_universe_table(self, filter_term=""):
        df_to_display = self.market_universe_df
        if filter_term:
            df_to_display = self.market_universe_df[
                self.market_universe_df['Company'].str.lower().str.contains(filter_term) |
                self.market_universe_df['Ticker'].str.lower().str.contains(filter_term)
            ]
        
        if df_to_display.empty:
            self.company_results_display_area.value = "<p style='text-align:center; color:red;'>No companies found matching your search.</p>"
            return

        table_html = "<table><thead><tr><th>Ticker</th><th>Company</th><th>Sector</th><th>Risk Classification</th><th>Market Cap (Cr)</th></tr></thead><tbody>"
        for _, row in df_to_display.head(20).iterrows(): # Display top 20 results or all if less
            mc_cr = f"₹{row.get('Market Cap (Cr)', 0):,.0f}" if pd.notna(row.get('Market Cap (Cr)')) else 'N/A'
            risk_class = row.get('Risk Classification', 'N/A')
            risk_color = '#27ae60' if risk_class == 'Low Risk' else '#f39c12' if risk_class == 'Medium Risk' else '#e74c3c'

            table_html += f"""
                <tr><td>{row.get('Ticker', 'N/A')}</td><td>{row.get('Company', 'N/A')}</td>
                <td>{row.get('Sector', 'N/A')}</td>
                <td style='color:{risk_color}; font-weight:bold;'>{risk_class}</td>
                <td style='text-align:right;'>{mc_cr}</td></tr>"""
        table_html += "</tbody></table>"
        if len(df_to_display) > 20:
            table_html += "<p style='text-align:center; font-style:italic;'>Showing top 20 results. Refine search for more specific companies.</p>"
        self.company_results_display_area.value = table_html


    def _create_stress_testing_tab(self):
        s_results = self.stress_test_results
        impact_summary = s_results.get('portfolio_impact_summary', [])
        
        summary_html = f"""
        <div class='action-card' style='background: linear-gradient(135deg, #ff758c 0%, #ff7eb3 100%); border-left: 5px solid #ff4757;'>
            <h2 style='margin-top:0;'>Portfolio Stress Test Summary (from Step 4)</h2>
            <p><strong>Client:</strong> {s_results.get('client_name', 'N/A')}</p>
            <p><strong>Risk Profile Considered:</strong> {s_results.get('client_risk_profile', 'N/A')}</p>
            <p><strong>Total Investment Tested:</strong> ₹{s_results.get('total_investment', 0):,}</p>
        </div>"""
        
        scenarios_html = "<div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(280px, 1fr)); gap: 15px; margin-top:15px;'>"
        if impact_summary:
            for res in impact_summary:
                scenarios_html += f"""
                <div class='metrics-card' style='border-left: 4px solid #c0392b;'>
                    <h4 style='color:#c0392b; margin-top:0;'>{res.get('scenario','N/A Scenario')}</h4>
                    <p><strong>Est. Portfolio Drawdown:</strong> <span style='color:red;'>{res.get('portfolio_drawdown',0):.1f}%</span></p>
                    <p><strong>Potential Loss Value:</strong> <span style='color:red;'>₹{res.get('potential_loss',0):,.0f}</span></p>
                    <p><strong>Est. Recovery:</strong> {res.get('recovery_days',0)//30} months</p>
                </div>"""
        else:
            scenarios_html = "<p style='text-align:center; font-style:italic;'>Stress test results not available from Step 4.</p>"
        scenarios_html += "</div>"

        # Key Learnings from paste-6.txt (adapted)
        learnings_html = """
        <div class='client-card' style='margin-top:20px;'>
        <h4>💡 Key Learnings & Expectations</h4>
        <ul>
            <li>Markets can be unpredictable, and downturns (like stormy weather) are normal.</li>
            <li>Historically, markets have always recovered and reached new highs over the long term.</li>
            <li>Staying invested and not panicking during dips is often the best strategy.</li>
            <li>Your specific portfolio is designed to align with your comfort for such 'bumps'.</li>
            <li>Regular reviews can help adjust to changing market conditions or life events.</li>
        </ul>
        </div>
        """
        self.stress_testing_tab_content = widgets.VBox([
            widgets.HTML(summary_html), widgets.HTML(scenarios_html), widgets.HTML(learnings_html)
        ])

    def _create_action_plan_tab(self):
        client_name = self.client_profile_data.get('client_name', 'Friend')
        total_investment = self.portfolio_data.get('total_investment', 0)
        num_holdings = len(self.portfolio_data.get('portfolio_breakdown', []))

        action_plan_html = f"""
        <div class='action-card'>
            <h2 style='margin-top:0;'>Your Personalized Investment Action Plan</h2>
            <p>Dear {client_name}, based on our comprehensive discussion and analysis, here's a suggested path forward:</p>
            
            <h3>Phase 1: Foundation & Setup (Next 1-2 Weeks)</h3>
            <ol>
                <li><strong>✅ Finalize Investment Amount:</strong> Confirm the total amount of ₹{total_investment:,.0f} (or adjust as discussed).</li>
                <li><strong>✅ Open/Verify Demat & Trading Account:</strong> Ensure your account with a preferred broker is active and funded.</li>
                <li><strong>✅ Complete KYC Formalities:</strong> If not already done, complete all Know Your Customer requirements.</li>
                <li><strong>✅ Review Emergency Fund:</strong> Current ₹{self.client_profile_data.get('income_expenses',{}).get('emergency_fund',0):,}. Ensure it covers 6-12 months of essential expenses.</li>
            </ol>

            <h3>Phase 2: Portfolio Implementation (Next 2-4 Weeks)</h3>
            <ol>
                <li><strong>✅ Staggered Investment:</strong> Consider investing the total amount over a few weeks/months (e.g., 3-4 tranches) if market is volatile, or lump sum if confident.</li>
                <li><strong>✅ Execute Trades:</strong> Place orders to buy the {num_holdings} recommended stocks as per their allocation percentages.
                    <ul>"""
        
        portfolio_breakdown = self.portfolio_data.get('portfolio_breakdown', [])
        for stock in portfolio_breakdown[:3]: # Show first 3 for brevity in action plan
            action_plan_html += f"<li>Buy {stock.get('company_name','N/A')} (approx. ₹{stock.get('investment_amount',0):,.0f})</li>"
        if len(portfolio_breakdown) > 3:
             action_plan_html += "<li>... and other recommended holdings.</li>"

        action_plan_html += f"""
                    </ul>
                </li>
                <li><strong>✅ Set Up SIPs (Systematic Investment Plans) - Optional:</strong>
                    If applicable, for any recommended Mutual Funds or to add regularly to this equity portfolio.
                    Suggested monthly SIP: ₹{(self.client_profile_data.get('income_expenses',{}).get('monthly_income',0) - self.client_profile_data.get('income_expenses',{}).get('monthly_expenses',0)) * 0.2 :,.0f} (e.g. 20% of surplus).
                </li>
            </ol>

            <h3>Phase 3: Monitoring & Review (Ongoing)</h3>
            <ol>
                <li><strong>✅ Portfolio Tracking:</strong> Use your broker's platform or tools like TickerTape/ValueResearch to monitor performance.</li>
                <li><strong>✅ Annual Review:</strong> Schedule a comprehensive portfolio review with your advisor at least once a year, or upon major life changes.</li>
                <li><strong>✅ Goal Tracking:</strong> Periodically check progress towards your financial goals.</li>
                <li><strong>✅ Stay Informed, Not Overwhelmed:</strong> Keep an eye on market news but avoid making impulsive decisions based on daily noise.</li>
                <li><strong>✅ Tax Planning:</strong> Consult on tax implications (e.g., LTCG, STCG) as your investments grow.</li>
            </ol>
            
            <div style='margin-top:20px; padding:15px; background-color:#e8f4fd; border-radius:8px;'>
            <p><strong>Important Note:</strong> This action plan is a guideline. Market conditions and personal circumstances can change. Always consult with your financial advisor before making significant investment decisions.</p>
            </div>
        </div>
        """
        self.action_plan_tab_content = widgets.VBox([widgets.HTML(action_plan_html)])

# --- Example of how to use this Step 5 class ---
# This part would typically be in your main Jupyter Notebook cell for Step 5.
# Ensure Step 0, 1, 2, 3 and 4 have run and set their respective global variables.

# Example: If you were running this cell directly after Step 4,
# 'current_client_profile', 'current_risk_profile', 'curated_companies',
# 'final_portfolio_step3', and 'stress_test_results_step4'
# should already exist in the global scope.

print("--- Initializing Step 5: Comprehensive Advisory Dashboard ---")
# Initialize Step 5's Dashboard. It will try to fetch global data from previous steps.
dashboard_instance = ComprehensiveAdvisoryDashboard()

# Display the Step 5 Dashboard interface.
display(dashboard_instance.main_container)


--- Initializing Step 5: Comprehensive Advisory Dashboard ---
